# Install Dependencies
Install additional packages needed to run SQL command in Jupyter cells and send them to a Postgres database server.

## PostgreSQL
Allows for a self-contained instance of PostresSQL for this notebook.

In [ ]:
%mamba install -y -c conda-forge postgresql=17 --quiet
!psql --version

## PostgreSQL Adapter for Python
Essential for connecting and interacting with PostgreSQL databases from Python code in Jupyter notebooks.

In [ ]:
%mamba install -c conda-forge psycopg2 -y --quiet
!python -c "import psycopg2; print(psycopg2.__version__)"

## Language Server Protocol for Jupyter
Enables code completion, hover documentation, and real-time syntax checking for multiple languages in Jupyter notebooks, including SQL.

In [ ]:
%mamba install -c conda-forge jupyterlab-lsp -y --quiet
!jupyter labextension list
!jupyter server extension list

## Language Server for Python
Provides real-time linting, code suggestions, and auto-completion in Jupyter notebooks for Python code.

In [ ]:
%mamba install -c conda-forge python-lsp-server -y --quiet
!pylsp --version

## SQL Magic Extension for Jupyter
Allows running SQL queries directly in Jupyter notebooks using SQL magic (%%sql). Enables seamless execution of SQL queries inside a Jupyter Notebook without switching to a database client.

In [ ]:
%mamba install -c conda-forge jupysql -y --quiet
!python -c "import sql; print(sql.__version__)"
!jupyter labextension list

## SQL Editor Extension
Enables SQL-specific features in Jupyter notebooks, such as:
* Syntax highlighting
* Query auto-completion
* Error checking

In [ ]:
%pip install jupyterlab_sql_editor --quiet
!jupyter labextension list

# Start PostgreSQL

## Define paths and logfile

In [1062]:
import os

DB_DIR = "mylocal_db"
LOGFILE = "postgres_log.txt"
PG_HBA_PATH = os.path.join(DB_DIR, "pg_hba.conf")

## Check PostgreSQL status

In [1063]:
!pg_ctl -D $DB_DIR status

pg_ctl: server is running (PID: 74777)
/Users/nashspence/miniforge3/envs/jupyter-env/bin/postgres "-D" "mylocal_db"


## Stop any existing PostgreSQL process

In [1064]:
!pg_ctl stop -D $DB_DIR -m fast || echo "Nothing to stop."

waiting for server to shut down.... done
server stopped


## Initialize the database if necessary

In [1065]:
import os

if not os.path.exists(DB_DIR) or not os.path.exists(os.path.join(DB_DIR, "PG_VERSION")):
    os.makedirs(DB_DIR, exist_ok=True)
    !initdb -D $DB_DIR

## Modify pg_hba.conf to allow trust authentication for user "notebook"

In [1066]:
with open(PG_HBA_PATH, "a") as pg_hba:
    pg_hba.write("\nlocal   all             notebook                                trust\n")
    pg_hba.write("host    all             notebook        127.0.0.1/32          trust\n")
    pg_hba.write("host    all             notebook        ::1/128               trust\n")

## Start PostgreSQL server

In [1067]:
import subprocess
import time
import atexit

def stop_postgres():
    !pg_ctl -D $DB_DIR stop

atexit.register(stop_postgres)

with open(LOGFILE, "a") as logfile:
    subprocess.Popen(["pg_ctl", "-D", DB_DIR, "-l", LOGFILE, "start"],
                     stdout=logfile, stderr=logfile)
time.sleep(5)

## Create the "notebook" user with full permissions

In [1068]:
!psql -d postgres -c "CREATE USER notebook WITH SUPERUSER;" || echo "User already exists."
!psql -d postgres -c "ALTER USER notebook WITH PASSWORD NULL;"

ERROR:  role "notebook" already exists
User already exists.
ALTER ROLE


# Connect

## Load & Configure SQLMagic

In [1069]:
%load_ext sql
%config SqlMagic.displaylimit = None
%config SqlMagic.named_parameters = "enabled"

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


displaylimit: Value None will be treated as 0 (no limit)

## Create Fresh sahuagin Database

In [1070]:
!psql -U notebook -d postgres -c "SELECT pg_terminate_backend(pid) FROM pg_stat_activity WHERE datname = 'sahuagin';"
!psql -U notebook -d postgres -c "DROP DATABASE IF EXISTS sahuagin;"
!psql -U notebook -d postgres -c "CREATE DATABASE sahuagin;"

 pg_terminate_backend 
----------------------
(0 rows)

DROP DATABASE
CREATE DATABASE


## Connect to sahuagin Database

In [1071]:
%sql postgresql://notebook@localhost:5432/sahuagin

# Create Tables & Types

## Attribute Type

In [1072]:
%%sql

DROP TYPE IF EXISTS attribute_type CASCADE;
CREATE TYPE attribute_type AS ENUM ('discrete','continuous');

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

/Users/nashspence/miniforge3/envs/jupyter-env/lib/python3.11/site-packages/sql/connection/connection.py:899: JupySQLRollbackPerformed: Server closed connection. JupySQL executed a ROLLBACK operation.
  warnings.warn(


++
||
++
++

## Attribute

In [1073]:
%%sql

DROP TABLE IF EXISTS attribute CASCADE;
CREATE TABLE attribute (
    id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    name varchar(255) NOT NULL UNIQUE,
    type attribute_type NOT NULL,
    decimals integer,
    max_value double precision,
    min_value double precision,
    mode_value double precision,
    concentration double precision,
    skew double precision,
    units varchar(255),
    CHECK (
        max_value IS NULL
        OR min_value IS NULL
        OR max_value >= min_value
    ),
    CHECK (
        (
            type = 'discrete'
            AND decimals IS NULL
            AND max_value IS NULL
            AND min_value IS NULL
            AND mode_value IS NULL
            AND concentration IS NULL
            AND skew IS NULL
            AND units IS NULL
        )
        OR (
            type = 'continuous'
            AND decimals IS NOT NULL
            AND max_value IS NOT NULL
            AND min_value IS NOT NULL
            AND mode_value IS NOT NULL
            AND concentration IS NOT NULL
            AND skew IS NOT NULL
        )
    )
);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Span

In [1074]:
%%sql

DROP TABLE IF EXISTS span CASCADE;
CREATE TABLE span (
    id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    attribute_id integer NOT NULL,
    label varchar(255) NOT NULL,
    type attribute_type NOT NULL,
    is_pinned boolean,
    weight double precision,
    max_value double precision,
    min_value double precision,
    CONSTRAINT uq_span_label_within_attribute UNIQUE (attribute_id, type, label),
    CONSTRAINT fk_span_to_attribute FOREIGN KEY (attribute_id) REFERENCES attribute (id) ON DELETE CASCADE,
    CHECK (
        max_value IS NULL
        OR min_value IS NULL
        OR max_value >= min_value
    ),
    CHECK (
        (
            type = 'discrete'
            AND is_pinned IS NOT NULL
            AND weight IS NOT NULL
            AND max_value IS NULL
            AND min_value IS NULL
        )
        OR (
            type = 'continuous'
            AND is_pinned IS NULL
            AND weight IS NULL
            AND max_value IS NOT NULL
            AND min_value IS NOT NULL
        )
    )
);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Variant

In [1075]:
%%sql

DROP TABLE IF EXISTS variant CASCADE;
CREATE TABLE variant (
    id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    name varchar(255) NOT NULL UNIQUE
);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Attribute Reference

In [1076]:
%%sql

DROP TABLE IF EXISTS variant_attr CASCADE;
CREATE TABLE variant_attr (
    id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    attribute_id integer NOT NULL,
    name varchar(255) NOT NULL,
    causation_index integer NOT NULL,
    variant_id integer NOT NULL,
    CONSTRAINT unique_variant_name UNIQUE (variant_id, name),
    CONSTRAINT unique_variant_index UNIQUE (variant_id, causation_index),
    CONSTRAINT fk_variant_attr_attr FOREIGN KEY (attribute_id) REFERENCES attribute (id) ON DELETE CASCADE,
    CONSTRAINT fk_variant_attr_variant FOREIGN KEY (variant_id) REFERENCES variant (id) ON DELETE CASCADE
);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Subvariant Span

In [1077]:
%%sql

DROP TABLE IF EXISTS subvariant_span CASCADE;
CREATE TABLE subvariant_span (
    id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    span_id integer NOT NULL,
    variant_id integer NOT NULL,
    CONSTRAINT unique_variant_span UNIQUE (variant_id, span_id),
    CONSTRAINT fk_varattr_span_span FOREIGN KEY (span_id) REFERENCES span (id) ON DELETE CASCADE,
    CONSTRAINT fk_varattr_span_variant FOREIGN KEY (variant_id) REFERENCES variant (id) ON DELETE CASCADE
);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Possible Attribute

In [1078]:
%%sql

DROP TABLE IF EXISTS possible_attr CASCADE;
CREATE TABLE possible_attr (
    id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    variant_attr_id integer NOT NULL,
    subvariant_span_id integer,
    parent_possible_attr_id integer,
    CONSTRAINT unique_possible_attr UNIQUE (parent_possible_attr_id, subvariant_span_id, variant_attr_id),
    CONSTRAINT fk_possible_attr_va FOREIGN KEY (variant_attr_id) REFERENCES variant_attr (id) ON DELETE CASCADE,
    CONSTRAINT fk_possible_attr_ss FOREIGN KEY (subvariant_span_id) REFERENCES subvariant_span (id) ON DELETE CASCADE,
    CONSTRAINT fk_possible_attr_parent FOREIGN KEY (parent_possible_attr_id) REFERENCES possible_attr (id) ON DELETE CASCADE
);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Effect

In [1079]:
%%sql

DROP TABLE IF EXISTS effect CASCADE;
CREATE TABLE effect (
    id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    name varchar(255) NOT NULL,
    variant_id integer NOT NULL,
    span_id integer NOT NULL,
    to_modify_possible_attr_id integer NOT NULL,
    activating_possible_attr_id integer NOT NULL,
    CONSTRAINT unique_effect_name_within_variant UNIQUE (name, variant_id),
    CONSTRAINT fk_effect_variant FOREIGN KEY (variant_id) REFERENCES variant (id) ON DELETE CASCADE,
    CONSTRAINT fk_effect_span FOREIGN KEY (span_id) REFERENCES span (id) ON DELETE CASCADE,
    CONSTRAINT fk_effect_psattr_to_modify FOREIGN KEY (to_modify_possible_attr_id) REFERENCES possible_attr (id) ON DELETE CASCADE,
    CONSTRAINT fk_effect_psattr_activating FOREIGN KEY (activating_possible_attr_id) REFERENCES possible_attr (id) ON DELETE CASCADE
);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Delta Parameter Effect

In [1080]:
%%sql

DROP TABLE IF EXISTS variation_continuous_attr CASCADE;
CREATE TABLE variation_continuous_attr (    
    id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    effect_id integer NOT NULL,
    delta_mode double precision NOT NULL,
    delta_conc double precision NOT NULL,
    delta_skew double precision NOT NULL,
    CONSTRAINT fk_var_continuous_attr_effect FOREIGN KEY (effect_id) REFERENCES effect (id) ON DELETE CASCADE
);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Additional Possibility Effect

In [1081]:
%%sql

DROP TABLE IF EXISTS variation_activated_span CASCADE;
CREATE TABLE variation_activated_span (
    id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    effect_id integer NOT NULL,
    span_id integer NOT NULL,
    CONSTRAINT fk_var_activated_span_span FOREIGN KEY (span_id) REFERENCES span (id) ON DELETE CASCADE,
    CONSTRAINT fk_var_activated_span_effect FOREIGN KEY (effect_id) REFERENCES effect (id) ON DELETE CASCADE
);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Delta Weight Effect

In [1082]:
%%sql

DROP TABLE IF EXISTS variation_delta_weight CASCADE;
CREATE TABLE variation_delta_weight (
    id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    effect_id integer NOT NULL,
    span_id integer NOT NULL,
    delta_weight double precision NOT NULL,
    CONSTRAINT fk_var_delta_weight_span FOREIGN KEY (span_id) REFERENCES span (id) ON DELETE CASCADE,
    CONSTRAINT fk_var_delta_weight_effect FOREIGN KEY (effect_id) REFERENCES effect (id) ON DELETE CASCADE
);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Removed Possibility Effect

In [1083]:
%%sql

DROP TABLE IF EXISTS variation_inactive_span CASCADE;
CREATE TABLE variation_inactive_span (
    id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    effect_id integer NOT NULL,
    span_id integer NOT NULL,
    CONSTRAINT fk_var_inactive_span_span FOREIGN KEY (span_id) REFERENCES span (id) ON DELETE CASCADE,
    CONSTRAINT fk_var_inactive_span_effect FOREIGN KEY (effect_id) REFERENCES effect (id) ON DELETE CASCADE
);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Entity

In [1084]:
%%sql

DROP TABLE IF EXISTS entity CASCADE;
CREATE TABLE entity (
    id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    variant_id integer NOT NULL,
    name varchar(255) NOT NULL UNIQUE,
    CONSTRAINT fk_entity_variant FOREIGN KEY (variant_id) REFERENCES variant (id) ON DELETE CASCADE
);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Entity Group

In [1085]:
%%sql

DROP TABLE IF EXISTS entity_group CASCADE;
CREATE TABLE entity_group (
    id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    name varchar(255) NOT NULL UNIQUE
);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

/Users/nashspence/miniforge3/envs/jupyter-env/lib/python3.11/site-packages/IPython/core/displayhook.py:281: UserWarning: Output cache limit (currently 1000 entries) hit.
Flushing oldest 200 entries.
  warn('Output cache limit (currently {sz} entries) hit.\n'


++
||
++
++

## Entity Group Link

In [1086]:
%%sql

DROP TABLE IF EXISTS entity_group_link CASCADE;
CREATE TABLE entity_group_link (
    id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    entity_id integer NOT NULL,
    entity_group_id integer NOT NULL,
    CONSTRAINT unique_link_entity_group UNIQUE (entity_group_id, entity_id),
    CONSTRAINT fk_entity FOREIGN KEY (entity_id) REFERENCES entity (id) ON DELETE CASCADE,
    CONSTRAINT fk_entity_group FOREIGN KEY (entity_group_id) REFERENCES entity_group (id) ON DELETE CASCADE
);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Entity State

In [1087]:
%%sql

DROP TABLE IF EXISTS entity_state CASCADE;
CREATE TABLE entity_state (
    id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    entity_id integer NOT NULL,
    time double precision NOT NULL,
    CONSTRAINT unique_entity_time UNIQUE (entity_id, time),
    CONSTRAINT fk_entity FOREIGN KEY (entity_id) REFERENCES entity (id) ON DELETE CASCADE
);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Attribute Value

In [1088]:
%%sql

DROP TABLE IF EXISTS entity_varattr_value CASCADE;
CREATE TABLE entity_varattr_value (
    id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    entity_state_id integer NOT NULL,
    numeric_value double precision,
    span_id integer,
    possible_attr_id integer NOT NULL,
    CONSTRAINT unique_state_possible_attr UNIQUE (entity_state_id, possible_attr_id),
    CONSTRAINT fk_evav_entity_state FOREIGN KEY (entity_state_id) REFERENCES entity_state (id) ON DELETE CASCADE,
    CONSTRAINT fk_evav_span FOREIGN KEY (span_id) REFERENCES span (id) ON DELETE CASCADE,
    CONSTRAINT fk_evav_possible_attr FOREIGN KEY (possible_attr_id) REFERENCES possible_attr (id) ON DELETE CASCADE
);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Attribute Value Lock

In [1089]:
%%sql

DROP TABLE IF EXISTS evav_lock CASCADE;
CREATE TABLE evav_lock (
    id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    locked_evav_id integer NOT NULL,
    locking_evav_id integer,
    CONSTRAINT unique_evav_lock UNIQUE (locking_evav_id, locked_evav_id),
    CONSTRAINT fk_evav_lock_locked_evav FOREIGN KEY (locked_evav_id) REFERENCES entity_varattr_value (id) ON DELETE CASCADE,
    CONSTRAINT fk_evav_lock_locking_evav FOREIGN KEY (locking_evav_id) REFERENCES entity_varattr_value (id) ON DELETE CASCADE
);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

# Triggers

## Delete Orphan possible_attr

In [1090]:
%%sql

CREATE OR REPLACE FUNCTION check_delete_possible_attr_from_effect()
RETURNS TRIGGER AS $$
DECLARE
    orphan_count INTEGER;
BEGIN
    -- Check the to_modify_possible_attr_id column
    IF OLD.to_modify_possible_attr_id IS NOT NULL THEN
        SELECT
            (SELECT COUNT(*) FROM effect WHERE to_modify_possible_attr_id = OLD.to_modify_possible_attr_id)
          + (SELECT COUNT(*) FROM effect WHERE activating_possible_attr_id = OLD.to_modify_possible_attr_id)
          + (SELECT COUNT(*) FROM entity_varattr_value WHERE possible_attr_id = OLD.to_modify_possible_attr_id)
          INTO orphan_count;
          
        IF orphan_count = 0 THEN
            DELETE FROM possible_attr WHERE id = OLD.to_modify_possible_attr_id;
        END IF;
    END IF;
    
    -- Check the activating_possible_attr_id column (if different)
    IF OLD.activating_possible_attr_id IS NOT NULL
       AND (OLD.activating_possible_attr_id <> OLD.to_modify_possible_attr_id) THEN
        SELECT
            (SELECT COUNT(*) FROM effect WHERE to_modify_possible_attr_id = OLD.activating_possible_attr_id)
          + (SELECT COUNT(*) FROM effect WHERE activating_possible_attr_id = OLD.activating_possible_attr_id)
          + (SELECT COUNT(*) FROM entity_varattr_value WHERE possible_attr_id = OLD.activating_possible_attr_id)
          INTO orphan_count;
          
        IF orphan_count = 0 THEN
            DELETE FROM possible_attr WHERE id = OLD.activating_possible_attr_id;
        END IF;
    END IF;
    
    RETURN NULL;
END;
$$ LANGUAGE plpgsql;

CREATE OR REPLACE FUNCTION check_delete_possible_attr_from_evav()
RETURNS TRIGGER AS $$
DECLARE
    orphan_count INTEGER;
BEGIN
    SELECT
        (SELECT COUNT(*) FROM effect 
           WHERE to_modify_possible_attr_id = OLD.possible_attr_id
              OR activating_possible_attr_id = OLD.possible_attr_id)
      + (SELECT COUNT(*) FROM entity_varattr_value WHERE possible_attr_id = OLD.possible_attr_id)
      INTO orphan_count;
      
    IF orphan_count = 0 THEN
        DELETE FROM possible_attr WHERE id = OLD.possible_attr_id;
    END IF;
    
    RETURN NULL;
END;
$$ LANGUAGE plpgsql;

CREATE OR REPLACE TRIGGER trg_check_delete_possible_attr_from_effect
AFTER DELETE ON effect
FOR EACH ROW
EXECUTE FUNCTION check_delete_possible_attr_from_effect();

CREATE OR REPLACE TRIGGER trg_check_delete_possible_attr_from_evav
AFTER DELETE ON entity_varattr_value
FOR EACH ROW
EXECUTE FUNCTION check_delete_possible_attr_from_evav();

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Delete Orphan Span

In [1091]:
%%sql

CREATE OR REPLACE FUNCTION delete_orphaned_span() RETURNS TRIGGER AS $$
BEGIN
    DELETE FROM span WHERE id = OLD.span_id;
    RETURN OLD;
END;
$$ LANGUAGE plpgsql;

CREATE OR REPLACE TRIGGER trg_delete_span_after_activation_delete
AFTER DELETE ON variation_activated_span
FOR EACH ROW
EXECUTE FUNCTION delete_orphaned_span();

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

# Create Indexes

## idx_attribute_name

In [1092]:
%%sql

DROP INDEX IF EXISTS idx_attribute_name;
CREATE INDEX idx_attribute_name ON attribute (name);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_span_attribute_id

In [1093]:
%%sql

DROP INDEX IF EXISTS idx_span_attribute_id;
CREATE INDEX idx_span_attribute_id ON span (attribute_id);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_span_attribute_type

In [1094]:
%%sql

DROP INDEX IF EXISTS idx_span_attribute_type;
CREATE INDEX idx_span_attribute_type ON span(attribute_id, type, id);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_span_attr_type

In [1095]:
%%sql

DROP INDEX IF EXISTS idx_span_attr_type;
CREATE INDEX idx_span_attr_type ON span(attribute_id, type, min_value, max_value);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_span_attr_type_pinned

In [1096]:
%%sql

DROP INDEX IF EXISTS idx_span_attr_type_pinned;
CREATE INDEX idx_span_attr_type_pinned ON span(attribute_id, type, is_pinned);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_span_attr_type_pinned_wl

In [1097]:
%%sql

DROP INDEX IF EXISTS idx_span_attr_type_pinned_wl;
CREATE INDEX idx_span_attr_type_pinned_wl ON span(attribute_id, type, is_pinned, weight, label);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_variant_name

In [1098]:
%%sql

DROP INDEX IF EXISTS idx_variant_name;
CREATE INDEX idx_variant_name ON variant (name);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_variant_attr_attribute

In [1099]:
%%sql

DROP INDEX IF EXISTS idx_variant_attr_attribute;
CREATE INDEX idx_variant_attr_attribute ON variant_attr (attribute_id);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_variant_attr_variant

In [1100]:
%%sql

DROP INDEX IF EXISTS idx_variant_attr_variant;
CREATE INDEX idx_variant_attr_variant ON variant_attr (variant_id);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_variant_attr_variant_causation

In [1101]:
%%sql

DROP INDEX IF EXISTS idx_variant_attr_variant_causation;
CREATE INDEX idx_variant_attr_variant_causation ON variant_attr (variant_id, causation_index);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_subvariant_span_span_id

In [1102]:
%%sql

DROP INDEX IF EXISTS idx_subvariant_span_span_id;
CREATE INDEX idx_subvariant_span_span_id ON subvariant_span (span_id);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_possible_attr_variant_attr

In [1103]:
%%sql

DROP INDEX IF EXISTS idx_possible_attr_variant_attr;
CREATE INDEX idx_possible_attr_variant_attr ON possible_attr (variant_attr_id);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_possible_attr_vaid

In [1104]:
%%sql

DROP INDEX IF EXISTS idx_possible_attr_vaid;
CREATE INDEX idx_possible_attr_vaid ON possible_attr (variant_attr_id, id);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_possible_attr_parent

In [1105]:
%%sql

DROP INDEX IF EXISTS idx_possible_attr_parent;
CREATE INDEX idx_possible_attr_parent ON possible_attr(parent_possible_attr_id);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_uq_possible_attr

In [1106]:
%%sql

DROP INDEX IF EXISTS idx_uq_possible_attr;
CREATE INDEX idx_uq_possible_attr ON possible_attr (
    variant_attr_id,
    subvariant_span_id
);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_possible_attr_va_pss

In [1107]:
%%sql

DROP INDEX IF EXISTS idx_uq_possible_attr;
CREATE INDEX idx_possible_attr_va_pss
  ON possible_attr(variant_attr_id, parent_possible_attr_id, subvariant_span_id);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_var_cont_variation

In [1108]:
%%sql

DROP INDEX IF EXISTS idx_var_cont_variation;
CREATE INDEX idx_var_cont_variation ON variation_continuous_attr (effect_id);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_effect_activating_psattr

In [1109]:
%%sql

DROP INDEX IF EXISTS idx_effect_activating_psattr;
CREATE INDEX idx_effect_activating_psattr ON effect(activating_possible_attr_id);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_effect_to_modify_psattr

In [1110]:
%%sql

DROP INDEX IF EXISTS idx_effect_to_modify_psattr;
CREATE INDEX idx_effect_to_modify_psattr ON effect(to_modify_possible_attr_id);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_variation_activated_span_effect

In [1111]:
%%sql

DROP INDEX IF EXISTS idx_variation_activated_span_effect;
CREATE INDEX idx_variation_activated_span_effect
  ON variation_activated_span(effect_id);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_variation_activated_span_varid_spanid

In [1112]:
%%sql

DROP INDEX IF EXISTS idx_variation_activated_span_effid_spanid;
CREATE INDEX idx_variation_activated_span_effid_spanid ON variation_activated_span (effect_id, span_id);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_variation_inactive_span_effect

In [1113]:
%%sql

DROP INDEX IF EXISTS idx_variation_inactive_span_effect;
CREATE INDEX idx_variation_inactive_span_effect
  ON variation_inactive_span(effect_id);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_variation_inactive

In [1114]:
%%sql

DROP INDEX IF EXISTS idx_variation_inactive;
CREATE INDEX idx_variation_inactive ON variation_inactive_span (effect_id, span_id);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_variation_delta_weight_effect

In [1115]:
%%sql

DROP INDEX IF EXISTS idx_variation_delta_weight_effect;
CREATE INDEX idx_variation_delta_weight_effect
  ON variation_delta_weight(effect_id);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_variation_delta_weight_varid_spanid

In [1116]:
%%sql

DROP INDEX IF EXISTS idx_variation_delta_weight_effid_spanid;
CREATE INDEX idx_variation_delta_weight_effid_spanid ON variation_delta_weight (effect_id, span_id);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_variation_delta

In [1117]:
%%sql

DROP INDEX IF EXISTS idx_variation_delta;
CREATE INDEX idx_variation_delta ON variation_delta_weight (
    effect_id,
    span_id,
    delta_weight
);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_entity_state_entity_id

In [1118]:
%%sql

DROP INDEX IF EXISTS idx_entity_state_entity_id;
CREATE INDEX idx_entity_state_entity_id ON entity_state (entity_id);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_entity_state_entity_time

In [1119]:
%%sql

DROP INDEX IF EXISTS idx_entity_state_entity_time;
CREATE INDEX idx_entity_state_entity_time ON entity_state (entity_id, time);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_entity_varattr_value_entity_state_id

In [1120]:
%%sql

DROP INDEX IF EXISTS idx_entity_varattr_value_entity_state_id;
CREATE INDEX idx_entity_varattr_value_entity_state_id ON entity_varattr_value (entity_state_id);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_evav_state_vaid

In [1121]:
%%sql

DROP INDEX IF EXISTS idx_evav_state_vaid;
CREATE INDEX idx_evav_state_vaid ON entity_varattr_value (
    entity_state_id,
    possible_attr_id
);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_evav_state_attr_span

In [1122]:
%%sql

DROP INDEX IF EXISTS idx_evav_state_attr_span;
CREATE INDEX idx_evav_state_attr_span ON entity_varattr_value (
    entity_state_id,
    possible_attr_id,
    span_id
);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_evav_lock_locked

In [1123]:
%%sql

DROP INDEX IF EXISTS idx_evav_lock_locked;
CREATE INDEX idx_evav_lock_locked ON evav_lock (locked_evav_id);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_evav_lock_locking

In [1124]:
%%sql

DROP INDEX IF EXISTS idx_evav_lock_locking;
CREATE INDEX idx_evav_lock_locking ON evav_lock (locking_evav_id);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_evav_lock_locked_locking

In [1125]:
%%sql

DROP INDEX IF EXISTS idx_evav_lock_locked_locking;
CREATE INDEX idx_evav_lock_locked_locking ON evav_lock (
    locked_evav_id,
    locking_evav_id
);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## idx_entity_group_link_group_id_entity_id

In [1126]:
%%sql

DROP INDEX IF EXISTS idx_entity_group_link_group_id_entity_id;
CREATE INDEX idx_entity_group_link_group_id_entity_id
  ON entity_group_link(entity_group_id, entity_id);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

# Debug

## Create Log Table

In [1127]:
%%sql

DROP TABLE IF EXISTS debug_log CASCADE;
CREATE TABLE debug_log (
    id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    log_time timestamp DEFAULT CURRENT_TIMESTAMP,
    procedure_name varchar(255),
    log_message text
);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Create Log Function

In [1128]:
%%sql

CREATE OR REPLACE FUNCTION debug_log(
    p_procedure_name varchar,
    p_log_message text
) RETURNS void
LANGUAGE plpgsql
AS $$
BEGIN
    INSERT INTO debug_log (procedure_name, log_message, log_time)
    VALUES (p_procedure_name, p_log_message, now());
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Clear Log Table

In [1129]:
%%sql

TRUNCATE TABLE debug_log RESTART IDENTITY CASCADE;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

# Create Functions & Procedures

## Discreate Attribute

### Add

In [1130]:
%%sql

CREATE OR REPLACE PROCEDURE add_discrete_attribute(
    in_name       VARCHAR(255),
    in_spans      JSON  -- JSON array: either [["span1", 10], ["span2", 20]] or ["span1", "span2"]
)
LANGUAGE plpgsql
AS $$
DECLARE
    new_attr_id       INTEGER;
    span_count        INTEGER;
    span_label        VARCHAR(255);
    span_weight       DOUBLE PRECISION;
    normalized_weight DOUBLE PRECISION;
    total_weight      DOUBLE PRECISION := 0;
    first_elem_type   TEXT;
    spans_json        JSON;
    i                 INTEGER;
BEGIN
    -- Create the new attribute
    INSERT INTO attribute (name, type)
      VALUES (in_name, 'discrete')
      RETURNING id INTO new_attr_id;

    IF in_spans IS NOT NULL THEN
        spans_json := in_spans::json;
        span_count := json_array_length(spans_json);

        IF span_count > 0 THEN
            first_elem_type := json_typeof(spans_json->0);
        ELSE
            first_elem_type := '';
        END IF;

        IF first_elem_type = 'array' THEN
            -- First pass: compute the total weight
            FOR i IN 0..span_count - 1 LOOP
                total_weight := total_weight + (((spans_json->i)->>1)::DOUBLE PRECISION);
            END LOOP;

            -- Second pass: insert spans with normalized weight
            FOR i IN 0..span_count - 1 LOOP
                span_label := (spans_json->i)->>0;
                span_weight := ((spans_json->i)->>1)::DOUBLE PRECISION;
                normalized_weight := span_weight / total_weight;
                INSERT INTO span (attribute_id, label, type, is_pinned, weight)
                  VALUES (new_attr_id, span_label, 'discrete', false, normalized_weight);
            END LOOP;
        ELSE
            -- For a JSON array of labels only, call the helper procedure for each span.
            FOR i IN 0..span_count - 1 LOOP
                span_label := spans_json->>i;
                CALL add_discrete_span(new_attr_id, span_label);
            END LOOP;
        END IF;
    END IF;
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

### Get

In [1131]:
%%sql

CREATE OR REPLACE FUNCTION get_discrete_attribute(p_attribute_name varchar)
RETURNS TABLE (
    attribute   varchar,
    type        attribute_type,
    span_label  varchar,
    span_weight double precision,
    is_pinned      boolean
)
LANGUAGE plpgsql
AS
$$
BEGIN
  RETURN QUERY
    SELECT 
      a.name,
      a.type,
      s.label,
      s.weight,
      s.is_pinned
    FROM attribute a
    JOIN span s ON s.attribute_id = a.id
    WHERE a.name = p_attribute_name
      AND NOT EXISTS (
          SELECT 1 
          FROM variation_activated_span ss 
          WHERE ss.span_id = s.id
      );
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

### Delete

In [1132]:
%%sql

CREATE OR REPLACE PROCEDURE delete_discrete_attribute(p_attribute_name varchar)
LANGUAGE plpgsql
AS
$$
BEGIN
  DELETE FROM attribute
  WHERE name = p_attribute_name
    AND type = 'discrete';
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Redistribute Unpinned Spans

In [1133]:
%%sql

CREATE OR REPLACE FUNCTION redistribute_unpinned_spans(
    p_attribute_id INTEGER,
    p_modified_span_id INTEGER,
    p_new_weight DOUBLE PRECISION,
    p_old_weight DOUBLE PRECISION
) RETURNS VOID
LANGUAGE plpgsql
AS $$
DECLARE
    v_pinned_weight DOUBLE PRECISION;
    v_target         DOUBLE PRECISION;
BEGIN
    -- Compute the total weight of pinned discrete spans.
    SELECT COALESCE(SUM(weight), 0)
      INTO v_pinned_weight
      FROM span
     WHERE attribute_id = p_attribute_id
       AND type = 'discrete'
       AND is_pinned = true;
       
    -- The available weight for all unpinned spans.
    v_target := 1.0 - v_pinned_weight;
    
    -- If there’s only one unpinned span (v_target equals p_old_weight), then
    -- the new weight must equal the entire target.
    IF (v_target - p_old_weight) = 0 THEN
       IF p_new_weight <> v_target THEN
           RAISE EXCEPTION 'Cannot change weight; only one unpinned span exists and its weight must be %', v_target;
       END IF;
    END IF;
    
    -- Update all unpinned discrete spans:
    -- • The modified span gets the new weight.
    -- • All others are scaled proportionally.
    UPDATE span
       SET weight = CASE 
                      WHEN id = p_modified_span_id THEN p_new_weight
                      ELSE weight * ((v_target - p_new_weight) / (v_target - p_old_weight))
                    END
     WHERE attribute_id = p_attribute_id
       AND type = 'discrete'
       AND is_pinned = false;
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Add Discrete Span to Attribute

In [1134]:
%%sql

CREATE OR REPLACE PROCEDURE add_disc_span_to_attr(
    p_attribute_id INTEGER,
    p_label VARCHAR(255)
)
LANGUAGE plpgsql
AS $$
DECLARE
    new_span_id     INTEGER;
    v_total_weight  DOUBLE PRECISION;
    v_pinned_weight DOUBLE PRECISION;
    v_target        DOUBLE PRECISION;
    v_count_old     INTEGER;
    candidate       DOUBLE PRECISION;
BEGIN
    -- Compute the total weight for all discrete spans for this attribute.
    SELECT COALESCE(SUM(weight), 0)
      INTO v_total_weight
      FROM span
     WHERE attribute_id = p_attribute_id
       AND type = 'discrete';

    -- If no spans exist, simply insert the first span with weight 1.
    IF v_total_weight = 0 THEN
        INSERT INTO span(attribute_id, label, type, is_pinned, weight)
        VALUES (p_attribute_id, p_label, 'discrete', false, 1.0);
        RETURN;
    END IF;
    
    -- Compute the total weight of pinned discrete spans.
    SELECT COALESCE(SUM(weight), 0)
      INTO v_pinned_weight
      FROM span
     WHERE attribute_id = p_attribute_id
       AND type = 'discrete'
       AND is_pinned = true;
    
    -- Calculate the available target weight for all unpinned spans.
    v_target := 1.0 - v_pinned_weight;
    IF v_target <= 0 THEN
        RAISE EXCEPTION 'No available weight for unpinned spans (v_target = %)', v_target;
    END IF;
    
    -- Count the existing non-pinned discrete spans.
    SELECT COUNT(*)
      INTO v_count_old
      FROM span
     WHERE attribute_id = p_attribute_id
       AND type = 'discrete'
       AND is_pinned = false;
    
    -- Insert the new unpinned span with a temporary weight of 0.
    INSERT INTO span(attribute_id, label, type, is_pinned, weight)
    VALUES (p_attribute_id, p_label, 'discrete', false, 0.0)
    RETURNING id INTO new_span_id;
    
    -- Compute the candidate weight for the new span.
    candidate := v_target / (v_count_old + 1);
    
    -- Redistribute the weights among all unpinned spans
    PERFORM redistribute_unpinned_spans(p_attribute_id, new_span_id, candidate, 0);
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Modify Discrete Span Weight

In [1135]:
%%sql

CREATE OR REPLACE PROCEDURE modify_disc_span_weight(
    p_span_id    INTEGER,
    p_new_weight DOUBLE PRECISION
)
LANGUAGE plpgsql
AS $$
DECLARE
    v_attribute_id INTEGER;
    v_current_weight DOUBLE PRECISION;
    v_is_pinned    BOOLEAN;
    v_type         TEXT;
    v_pinned_weight DOUBLE PRECISION;
    v_target        DOUBLE PRECISION;
    v_other_sum     DOUBLE PRECISION;
BEGIN
    -- Get the current span's details.
    SELECT attribute_id, weight, is_pinned, type
      INTO v_attribute_id, v_current_weight, v_is_pinned, v_type
      FROM span
     WHERE id = p_span_id;
     
    -- Ensure the span exists, is of type 'discrete' and is not pinned.
    IF NOT FOUND OR v_type <> 'discrete' OR v_is_pinned THEN
       RAISE EXCEPTION 'Span % not found or not a modifiable discrete span', p_span_id;
    END IF;
    
    -- Compute the total pinned weight for this attribute.
    SELECT COALESCE(SUM(weight), 0)
      INTO v_pinned_weight
      FROM span
     WHERE attribute_id = v_attribute_id
       AND type = 'discrete'
       AND is_pinned = true;
       
    v_target := 1.0 - v_pinned_weight;
    
    -- Validate the new weight is between 0 and the available target.
    IF p_new_weight < 0 OR p_new_weight > v_target THEN
       RAISE EXCEPTION 'Invalid new weight: % (must be between 0 and %)', p_new_weight, v_target;
    END IF;
    
    -- Compute the total weight for all other unpinned spans.
    SELECT COALESCE(SUM(weight), 0)
      INTO v_other_sum
      FROM span
     WHERE attribute_id = v_attribute_id
       AND type = 'discrete'
       AND is_pinned = false
       AND id <> p_span_id;
       
    IF v_other_sum = 0 THEN
       -- There is only one unpinned span. Its weight must equal the available target.
       IF p_new_weight <> v_target THEN
         RAISE EXCEPTION 'Only one unpinned span exists. Its weight must be %', v_target;
       ELSE
         UPDATE span SET weight = p_new_weight WHERE id = p_span_id;
       END IF;
    ELSE
       -- Use the helper function to redistribute weights.
       PERFORM redistribute_unpinned_spans(v_attribute_id, p_span_id, p_new_weight, v_current_weight);
    END IF;
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Continuous Attribute

### Add

In [1136]:
%%sql

CREATE OR REPLACE PROCEDURE add_continuous_attribute(
    p_name           VARCHAR(255),
    p_min_value      DOUBLE PRECISION,
    p_mode_value     DOUBLE PRECISION,
    p_max_value      DOUBLE PRECISION,
    p_concentration  DOUBLE PRECISION,
    p_skew           DOUBLE PRECISION,
    p_decimals       INTEGER,
    p_units          VARCHAR(255),
    p_spans          JSON DEFAULT NULL  -- JSON array shortcut for continuous spans
)
LANGUAGE plpgsql
AS $$
DECLARE
    v_attr_id   INTEGER;
    span_count  INTEGER;
    i           INTEGER;
    current_min DOUBLE PRECISION;
    new_max     DOUBLE PRECISION;
    span_label  VARCHAR(255);
    spans_json  JSON;
BEGIN
    -- Insert the continuous attribute record.
    INSERT INTO attribute
      (name, type, decimals, max_value, min_value,
       mode_value, concentration, skew, units)
    VALUES
      (p_name, 'continuous', p_decimals, p_max_value,
       p_min_value, p_mode_value, p_concentration, p_skew, p_units)
    RETURNING id INTO v_attr_id;
    
    IF p_spans IS NOT NULL THEN
        spans_json := p_spans::json;
        span_count := json_array_length(spans_json);
        current_min := p_min_value;
        
        FOR i IN 0..span_count - 1 LOOP
            -- Each JSON tuple is assumed to be [label, new_max_value].
            span_label := (spans_json->i)->>0;
            new_max := ((spans_json->i)->>1)::DOUBLE PRECISION;
            
            -- Validate that the new maximum is in the valid interval.
            IF new_max <= current_min OR new_max > p_max_value THEN
                RAISE EXCEPTION 'Invalid span range: new_max (%) is not in the valid interval (% - %)',
                    new_max, current_min, p_max_value;
            END IF;
            
            -- Insert the continuous span.
            INSERT INTO span(attribute_id, label, type, min_value, max_value)
            VALUES(v_attr_id, span_label, 'continuous', current_min, new_max);
            
            current_min := new_max;
        END LOOP;
        
        -- If the JSON array does not reach the attribute's overall max, raise an exception.
        IF current_min < p_max_value THEN
            RAISE EXCEPTION 'Incomplete continuous span definitions: final span max is % which is less than the attribute maximum %', current_min, p_max_value;
        END IF;
    END IF;
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

### Get

In [1137]:
%%sql

CREATE OR REPLACE FUNCTION get_continuous_attribute(p_attribute_name varchar)
RETURNS TABLE (
    name           varchar,
    type           attribute_type,
    min            double precision,
    mode           double precision,
    max            double precision,
    concentration  double precision,
    skew           double precision,
    units          varchar,
    decimals       integer,
    span_label     varchar,
    span_min       double precision,
    span_max       double precision
)
LANGUAGE plpgsql
AS
$$
BEGIN
  RETURN QUERY
    SELECT 
      a.name,
      a.type,
      a.min_value,
      a.mode_value,
      a.max_value,
      a.concentration,
      a.skew,
      a.units,
      a.decimals,
      s.label,
      s.min_value,
      s.max_value
    FROM attribute a
    LEFT JOIN span s ON s.attribute_id = a.id
    WHERE a.name = p_attribute_name
      AND a.type = 'continuous';
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

### Delete

In [1138]:
%%sql

CREATE OR REPLACE PROCEDURE delete_continuous_attribute(p_attribute_name varchar)
LANGUAGE plpgsql
AS
$$
BEGIN
  DELETE FROM attribute
  WHERE name = p_attribute_name
    AND type = 'continuous';
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Add Continuous Span to Attribute

In [1139]:
%%sql

CREATE OR REPLACE PROCEDURE add_cont_span_to_attr(
    p_attribute_id integer,
    p_label varchar(255),
    p_new_min double precision,
    p_new_max double precision
)
LANGUAGE plpgsql
AS $$
DECLARE
    v_attr_min double precision;
    v_attr_max double precision;
    v_count integer;
    v_first_id integer;
    v_first_min double precision;
    v_first_max double precision;
    v_last_id integer;
    v_last_min double precision;
    v_last_max double precision;
BEGIN
    -- Get attribute’s overall range (continuous only)
    SELECT min_value, max_value
      INTO v_attr_min, v_attr_max
      FROM attribute
     WHERE id = p_attribute_id
       AND type = 'continuous'
     LIMIT 1;
     
    IF v_attr_min IS NULL THEN
        RAISE EXCEPTION 'Attribute not found or not continuous';
    END IF;
    
    -- Validate requested span
    IF p_new_min < v_attr_min OR p_new_max > v_attr_max OR p_new_min >= p_new_max THEN
        RAISE EXCEPTION 'Invalid span range';
    END IF;
    
    -- If no continuous spans exist, insert one covering the full range
    SELECT COUNT(*) INTO v_count
      FROM span
     WHERE attribute_id = p_attribute_id
       AND type = 'continuous';
       
    IF v_count = 0 THEN
        INSERT INTO span(attribute_id, label, type, min_value, max_value)
        VALUES(p_attribute_id, p_label, 'continuous', v_attr_min, v_attr_max);
        RETURN;
    END IF;
    
    -- Adjust overlapping spans: first overlapping span (ordered by min_value)
    SELECT id, min_value, max_value
      INTO v_first_id, v_first_min, v_first_max
      FROM span
     WHERE attribute_id = p_attribute_id
       AND type = 'continuous'
       AND max_value > p_new_min
       AND min_value < p_new_max
     ORDER BY min_value ASC
     LIMIT 1;
     
    IF v_first_min < p_new_min THEN
        UPDATE span SET max_value = p_new_min WHERE id = v_first_id;
    END IF;
    
    -- Adjust overlapping spans: last overlapping span (ordered by max_value)
    SELECT id, min_value, max_value
      INTO v_last_id, v_last_min, v_last_max
      FROM span
     WHERE attribute_id = p_attribute_id
       AND type = 'continuous'
       AND max_value > p_new_min
       AND min_value < p_new_max
     ORDER BY max_value DESC
     LIMIT 1;
     
    IF v_last_max > p_new_max THEN
        UPDATE span SET min_value = p_new_max WHERE id = v_last_id;
    END IF;
    
    -- Remove spans fully covered by the new span
    DELETE FROM span
     WHERE attribute_id = p_attribute_id
       AND type = 'continuous'
       AND min_value >= p_new_min
       AND max_value <= p_new_max;
       
    INSERT INTO span(attribute_id, label, type, min_value, max_value)
    VALUES(p_attribute_id, p_label, 'continuous', p_new_min, p_new_max);
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Link Attribute to Variant

In [1140]:
%%sql

CREATE OR REPLACE PROCEDURE link_attr_to_variant(
    p_variant_id   INTEGER,
    p_attribute_id INTEGER,
    p_name         VARCHAR,
    p_position     INTEGER DEFAULT NULL
)
LANGUAGE plpgsql
AS $$
DECLARE
    v_count    INTEGER;
    v_position INTEGER;
BEGIN
    SELECT COUNT(*) INTO v_count
      FROM variant_attr
     WHERE variant_id = p_variant_id;
    
    IF p_position IS NULL THEN
        v_position := v_count;  -- append at end
    ELSE
        v_position := p_position;
        IF v_position < 0 THEN 
            v_position := 0;
        END IF;
        IF v_position > v_count THEN
            v_position := v_count;
        END IF;
        UPDATE variant_attr
           SET causation_index = causation_index + 1
         WHERE variant_id = p_variant_id
           AND causation_index >= v_position;
    END IF;

    INSERT INTO variant_attr(attribute_id, name, causation_index, variant_id)
    VALUES (p_attribute_id, p_name, v_position, p_variant_id);
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Variant

### Add

In [1141]:
%%sql

CREATE OR REPLACE PROCEDURE add_variant(
    in_variant_name VARCHAR,
    in_attr_keys    JSON
)
LANGUAGE plpgsql
AS $$
DECLARE
    new_variant_id INTEGER;
    i              INTEGER := 0;
    key_count      INTEGER;
    attr_id        INTEGER;
    attr_name      VARCHAR;
BEGIN
    INSERT INTO variant (name)
    VALUES (in_variant_name)
    RETURNING id INTO new_variant_id;

    IF in_attr_keys IS NOT NULL THEN
        key_count := json_array_length(in_attr_keys::json);
        WHILE i < key_count LOOP
            -- Get the attribute name from the JSON array at position i.
            attr_name := in_attr_keys::json ->> i;

            -- Look up the attribute id for the given name.
            SELECT id
              INTO attr_id
              FROM attribute
             WHERE name = attr_name
             LIMIT 1;

            IF attr_id IS NULL THEN
                RAISE EXCEPTION 'Attribute with name "%" not found', attr_name;
            END IF;

            -- Call the helper procedure with the looked up attribute id.
            CALL link_attr_to_variant(new_variant_id, attr_id, attr_name);
            i := i + 1;
        END LOOP;
    END IF;
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

### Get

In [1142]:
%%sql

CREATE OR REPLACE FUNCTION get_variant_page(
    p_variant_name VARCHAR,
    p_page INTEGER,
    p_page_size INTEGER
)
RETURNS TABLE (
    "index"     BIGINT,
    "address"   TEXT,
    activates   JSON,
    effected_by JSON
)
LANGUAGE plpgsql
AS $$
BEGIN
  RETURN QUERY
  WITH RECURSIVE variant_id_cte AS (
      -- Look up the variant’s id by its unique name.
      SELECT id
      FROM variant
      WHERE name = p_variant_name
  ),
  recursive_flattened AS (
      -- Base step: get all top‑level attributes for the variant.
      SELECT 
          va.id AS variant_attr_id,
          rva.subvariant_span_id,
          va.name AS attribute_name,
          va.causation_index,
          ARRAY[va.causation_index] AS flattened_path,
          /* Build the label as a JSON array containing a single inner array.
             Each inner array has the attribute name and, if available, the span label. */
          jsonb_build_array(
              CASE 
                WHEN s.label IS NOT NULL 
                  THEN jsonb_build_array(va.name, s.label)
                ELSE jsonb_build_array(va.name)
              END
          ) AS agg_label,
          va.variant_id AS current_variant_id,
          rva.id AS resolved_id
      FROM variant_attr va
      LEFT JOIN possible_attr rva 
         ON rva.variant_attr_id = va.id
            AND rva.parent_possible_attr_id IS NULL
      LEFT JOIN subvariant_span ss 
         ON ss.id = rva.subvariant_span_id
      LEFT JOIN span s 
         ON s.id = ss.span_id
      WHERE va.variant_id = (SELECT id FROM variant_id_cte)
      
      UNION ALL
      
      -- Recursive step: for each resolved node that activates a sub‑variant, get its children.
      SELECT 
          va_child.id AS variant_attr_id,
          rva_child.subvariant_span_id,
          va_child.name AS attribute_name,
          va_child.causation_index,
          rf.flattened_path || va_child.causation_index AS flattened_path,
          rf.agg_label || jsonb_build_array(
              CASE 
                WHEN s_child.label IS NOT NULL 
                  THEN jsonb_build_array(va_child.name, s_child.label)
                ELSE jsonb_build_array(va_child.name)
              END
          ) AS agg_label,
          va_child.variant_id AS current_variant_id,
          rva_child.id AS resolved_id
      FROM recursive_flattened rf
      -- Use the parent’s resolved record.
      JOIN possible_attr rva_parent
         ON rva_parent.id = rf.resolved_id
      -- The parent's associated subvariant_span holds the activated sub‑variant.
      JOIN subvariant_span ss_parent 
         ON ss_parent.id = rva_parent.subvariant_span_id
      -- A non‑null ss_parent.variant_id indicates the activated sub‑variant.
      JOIN variant_attr va_child 
         ON va_child.variant_id = ss_parent.variant_id
      LEFT JOIN possible_attr rva_child 
         ON rva_child.variant_attr_id = va_child.id
            AND rva_child.parent_possible_attr_id = rf.resolved_id
      LEFT JOIN subvariant_span ss_child 
         ON ss_child.id = rva_child.subvariant_span_id
      LEFT JOIN span s_child 
         ON s_child.id = ss_child.span_id
      WHERE ss_parent.variant_id IS NOT NULL
  ),
  ordered AS (
      SELECT 
          row_number() OVER (ORDER BY flattened_path) AS overall_index,
          agg_label,
          resolved_id,
          flattened_path
      FROM recursive_flattened
  ),
  paged AS (
      SELECT *
      FROM ordered
      ORDER BY flattened_path
      LIMIT p_page_size OFFSET (p_page - 1) * p_page_size
  )
  SELECT 
      p.overall_index AS "index",
      p.agg_label::text AS "address",
      (
         -- Effects activated by this node.
         SELECT COALESCE(json_agg(e.name ORDER BY e.name), '[]'::json)
         FROM effect e
         WHERE e.activating_possible_attr_id = p.resolved_id
      ) AS activates,
      (
         -- Effects applied to (modifying) this node.
         SELECT COALESCE(json_agg(e.name ORDER BY e.name), '[]'::json)
         FROM effect e
         WHERE e.to_modify_possible_attr_id = p.resolved_id
      ) AS effected_by
  FROM paged p
  ORDER BY p.overall_index;
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

### Delete

In [1143]:
%%sql

CREATE OR REPLACE PROCEDURE delete_variant(p_variant_name varchar)
LANGUAGE plpgsql
AS
$$
BEGIN
  DELETE FROM variant WHERE name = p_variant_name;
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Add Entity

In [1144]:
%%sql

CREATE OR REPLACE PROCEDURE add_entity(
    in_name VARCHAR(255),
    in_variant_id INT
)
LANGUAGE plpgsql
AS $$
BEGIN
    INSERT INTO entity(name, variant_id)
    VALUES (in_name, in_variant_id);
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## fn_same_slot

In [1145]:
%%sql

CREATE OR REPLACE FUNCTION fn_same_slot(
    p_candidate_id INTEGER,
    p_reference_id INTEGER
)
RETURNS BOOLEAN
LANGUAGE plpgsql
AS $$
DECLARE
    v_parent_candidate   INTEGER;
    v_variant_candidate  INTEGER;
    v_parent_reference   INTEGER;
    v_variant_reference  INTEGER;
BEGIN
    -- Get slot details for the candidate.
    SELECT parent_possible_attr_id, subvariant_span_id
      INTO v_parent_candidate, v_variant_candidate
      FROM possible_attr
     WHERE id = p_candidate_id;
    
    -- Get slot details for the reference.
    SELECT parent_possible_attr_id, subvariant_span_id
      INTO v_parent_reference, v_variant_reference
      FROM possible_attr
     WHERE id = p_reference_id;
    
    -- Return true if both the parent and subvariant_span values are equivalent.
    RETURN (v_parent_candidate IS NOT DISTINCT FROM v_parent_reference)
           AND (v_variant_candidate IS NOT DISTINCT FROM v_variant_reference);
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Roll Discrete Variant Attribute Value

In [1146]:
%%sql

CREATE OR REPLACE FUNCTION roll_discrete_varattr(
    p_possible_attr_id INTEGER,
    p_entity_state_id INTEGER,
    p_exclude_span_id INTEGER
) 
RETURNS INTEGER
LANGUAGE plpgsql
AS $$
DECLARE
    v_span_id INTEGER;
    v_max_double CONSTANT double precision := 1e308;
    v_variant_attr_id INTEGER;
    v_subvariant_span_id INTEGER;
BEGIN
    -- Retrieve the underlying variant attribute and associated span trigger.
    SELECT variant_attr_id, subvariant_span_id
      INTO v_variant_attr_id, v_subvariant_span_id
      FROM possible_attr
     WHERE id = p_possible_attr_id;
    
    WITH
    BaseSpans AS (
          SELECT s.id AS span_id, 
               va.id AS variant_attr_id, 
               NULL::INTEGER AS subvariant_span_id,
               s.weight AS base_weight
          FROM variant_attr va
          JOIN span s 
            ON s.attribute_id = va.attribute_id
          WHERE va.id = v_variant_attr_id
            AND s.type = 'discrete'
            AND (p_exclude_span_id = 0 OR s.id <> p_exclude_span_id)
            -- Exclude spans that were added for an effect
            AND s.id NOT IN (
                 SELECT ss.span_id
                   FROM variation_activated_span ss
            )
     ),
    ActiveEffects AS (
         /*
           Join to the effect table. We only want effects whose
           triggering span (effect.span_id) has already been rolled into
           the current entity_state.
         */
         SELECT e.id AS effect_id, 
                e.to_modify_possible_attr_id,
                e.activating_possible_attr_id
         FROM effect e
         WHERE e.variant_id = (
                 SELECT variant_id
                   FROM possible_attr
                  WHERE id = p_possible_attr_id
             )
           AND e.span_id IN (
                 SELECT span_id
                   FROM entity_varattr_value
                  WHERE entity_state_id = p_entity_state_id
             )
           AND e.to_modify_possible_attr_id IN (
                 SELECT pa.id
                   FROM possible_attr pa
                  WHERE pa.variant_attr_id = v_variant_attr_id
                    AND fn_same_slot(pa.id, p_possible_attr_id)
             )
    ),
    InactiveSpans AS (
         SELECT DISTINCT vis.span_id
         FROM variation_inactive_span vis  -- assuming these tables remain unchanged
         JOIN ActiveEffects ae ON ae.effect_id = vis.effect_id
    ), 
    ActivatedSpans AS (
         SELECT DISTINCT 
                ss.span_id, 
                ae.to_modify_possible_attr_id AS variant_attr_id, 
                NULL::INTEGER AS subvariant_span_id, 
                0.0 AS base_weight
         FROM variation_activated_span ss
         JOIN ActiveEffects ae ON ae.effect_id = ss.effect_id
    ), 
    DeltaWeights AS (
         SELECT vdw.span_id, 
                SUM(vdw.delta_weight) AS total_delta
         FROM variation_delta_weight vdw
         JOIN ActiveEffects ae ON ae.effect_id = vdw.effect_id
         GROUP BY vdw.span_id
    ), 
    AllRelevantSpans AS (
         SELECT b.span_id, b.variant_attr_id, b.subvariant_span_id, b.base_weight
         FROM BaseSpans b
         WHERE b.span_id NOT IN (SELECT span_id FROM InactiveSpans)
         UNION
         SELECT a.span_id, a.variant_attr_id, a.subvariant_span_id, a.base_weight
         FROM ActivatedSpans a
         WHERE a.span_id NOT IN (SELECT span_id FROM InactiveSpans)
    ), 
    FinalSpans AS (
         SELECT ars.span_id, 
                ars.variant_attr_id, 
                ars.subvariant_span_id,
                COALESCE(ars.base_weight, 0) + COALESCE(dw.total_delta, 0) AS effective_weight
         FROM AllRelevantSpans ars
         LEFT JOIN DeltaWeights dw ON dw.span_id = ars.span_id
    ),
    SpanCounts AS (
         SELECT ns.num_spans, 
                SUM(fs.effective_weight / ns.num_spans) AS total_weight
         FROM FinalSpans fs
         CROSS JOIN (
             SELECT COUNT(*)::double precision AS num_spans 
             FROM FinalSpans 
             WHERE effective_weight > 0
         ) ns
         WHERE fs.effective_weight > 0
         GROUP BY ns.num_spans
    ),
    AdjustedSpans AS (
         SELECT fs.span_id,
                fs.variant_attr_id,
                fs.subvariant_span_id,
                fs.effective_weight,
                sc.num_spans,
                sc.total_weight,
                CASE 
                  WHEN fs.effective_weight < 1 
                    THEN ceil((fs.effective_weight * v_max_double) / (sc.total_weight * sc.num_spans))
                  ELSE ceil((fs.effective_weight / (sc.total_weight * sc.num_spans)) * v_max_double)
                END AS adjusted_weight
         FROM FinalSpans fs
         CROSS JOIN SpanCounts sc
         WHERE fs.effective_weight > 0
    ),
    Running AS (
         SELECT span_id,
                variant_attr_id,
                subvariant_span_id,
                adjusted_weight,
                SUM(adjusted_weight) OVER (ORDER BY span_id) AS running_total
         FROM AdjustedSpans
    ),
    OrderedSpansCTE AS (
         SELECT span_id,
                variant_attr_id,
                subvariant_span_id,
                adjusted_weight AS contextual_weight,
                running_total,
                LAG(running_total, 1, 0) OVER (ORDER BY span_id) AS prev_running_total
         FROM Running
    ),
    TotalAdjusted AS (
         SELECT MAX(running_total) AS v_total_weight
         FROM OrderedSpansCTE
    ),
    Rng AS (
         SELECT t.v_total_weight,
                floor(random() * t.v_total_weight) AS v_random_pick
         FROM TotalAdjusted t
    )
    SELECT o.span_id
      INTO v_span_id
      FROM OrderedSpansCTE o, Rng
      WHERE Rng.v_random_pick >= o.prev_running_total
        AND Rng.v_random_pick < o.running_total
      LIMIT 1;

    RETURN v_span_id;
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Gamma Range

In [1147]:
%%sql

CREATE OR REPLACE FUNCTION gamma_rng(shape double precision)
RETURNS double precision
LANGUAGE plpgsql
AS $$
DECLARE
    d double precision;
    c double precision;
    x double precision;
    v double precision;
    u double precision;
BEGIN
    IF shape < 1 THEN
        -- Use the transformation: Gamma(shape) = Gamma(shape+1)*U^(1/shape)
        RETURN gamma_rng(shape + 1) * power(random(), 1.0 / shape);
    END IF;
    d := shape - 1.0/3.0;
    c := 1.0 / sqrt(9.0 * d);
    LOOP
        -- Generate a standard normal via Box–Muller:
        x := sqrt(-2 * ln(random())) * cos(2 * 3.141592653589793 * random());
        v := 1 + c * x;
        IF v <= 0 THEN
            CONTINUE;
        END IF;
        v := v * v * v;
        u := random();
        IF u < 1 - 0.0331 * x * x * x * x THEN
            RETURN d * v;
        END IF;
        IF ln(u) < 0.5 * x * x + d * (1 - v + ln(v)) THEN
            RETURN d * v;
        END IF;
    END LOOP;
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Generate Weighted Random Number

In [1148]:
%%sql

-- using a fast Marsaglia–Tsang method
CREATE OR REPLACE FUNCTION weighted_random(
    min_val double precision,                   -- Minimum of desired range.
    max_val double precision,                   -- Maximum of desired range (must be > min_val).
    mode_val double precision,                  -- Desired mode (peak) in the same units.
    skew double precision DEFAULT 0,            -- Skew adjustment; between -1 and 1 only, 0 = no skew.
    concentration double precision DEFAULT 4    -- Concentration (k); should be >2 for a proper unimodal shape.
) RETURNS double precision
LANGUAGE plpgsql
AS $$
DECLARE
    m_norm      double precision;  -- Normalized desired mode in [0,1].
    alpha       double precision;
    beta        double precision;
    x_val       double precision;
    y_val       double precision;
    beta_sample double precision;
BEGIN
    IF min_val >= max_val THEN
        RAISE EXCEPTION 'min_val must be less than max_val';
    END IF;
    IF concentration < 0 THEN
        RAISE EXCEPTION 'concentration must be non-negative';
    END IF;
    IF concentration <= 2 THEN
        -- When concentration <= 2 the Beta isn't unimodal; fallback to uniform.
        RETURN min_val + random() * (max_val - min_val);
    END IF;

    -- Normalize the desired mode to [0,1].
    m_norm := (mode_val - min_val) / (max_val - min_val);

    -- Parameterize α and β so that (α-1)/(α+β-2) equals m_norm.
    alpha := m_norm * (concentration - 2) + 1;
    beta  := (1 - m_norm) * (concentration - 2) + 1;

    -- Apply skew correction if desired.
    IF skew <> 0 THEN
        alpha := alpha * (1 + skew);
        beta  := beta  * (1 - skew);
    END IF;

    -- Generate a Beta(α,β) random variable via the Gamma method.
    x_val := gamma_rng(alpha);
    y_val := gamma_rng(beta);
    beta_sample := x_val / (x_val + y_val);

    -- Scale the [0,1] Beta sample to [min_val, max_val].
    RETURN min_val + beta_sample * (max_val - min_val);
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Roll Continuous Variant Attribute Value

In [1149]:
%%sql

CREATE OR REPLACE FUNCTION roll_continuous_varattr(
    p_possible_attr_id INTEGER,
    p_entity_state_id   INTEGER,
    p_exclude_span_id   INTEGER
)
RETURNS TABLE(chosen_span_id integer, chosen_value double precision)
LANGUAGE plpgsql
AS $$
DECLARE
    v_attribute_id         INTEGER;
    v_decimals             INTEGER;
    v_min                  DOUBLE PRECISION;
    v_max                  DOUBLE PRECISION;
    v_mode                 DOUBLE PRECISION;
    v_concentration        DOUBLE PRECISION;
    v_skew                 DOUBLE PRECISION;
    v_total_delta_mode     DOUBLE PRECISION := 0;
    v_total_delta_conc     DOUBLE PRECISION := 0;
    v_total_delta_skew     DOUBLE PRECISION := 0;
    v_eff_min              DOUBLE PRECISION;
    v_eff_max              DOUBLE PRECISION;
    v_eff_mode             DOUBLE PRECISION;
    v_result               DOUBLE PRECISION;
    v_clamped_result       DOUBLE PRECISION;
    v_chosen_span_id       INTEGER;
    dummy_eff_min          DOUBLE PRECISION;
    dummy_eff_max          DOUBLE PRECISION;
    v_variant_attr_id INTEGER;
BEGIN
    -- Retrieve attribute details from the possible_attr record.
    SELECT va.attribute_id, a.decimals, a.min_value, a.max_value, a.mode_value, 
           a.concentration, a.skew, va.id
      INTO v_attribute_id, v_decimals, v_min, v_max, v_mode, v_concentration, v_skew, v_variant_attr_id
      FROM possible_attr pa
      JOIN variant_attr va ON va.id = pa.variant_attr_id
      JOIN attribute a ON a.id = va.attribute_id
     WHERE pa.id = p_possible_attr_id
     LIMIT 1;

    IF v_attribute_id IS NULL THEN
        RAISE EXCEPTION 'No matching attribute found for possible_attr_id %', p_possible_attr_id;
    END IF;

    -- Sum continuous attribute delta values from effects that have been triggered.
    SELECT COALESCE(SUM(vca.delta_mode), 0),
           COALESCE(SUM(vca.delta_conc), 0),
           COALESCE(SUM(vca.delta_skew), 0)
      INTO v_total_delta_mode, v_total_delta_conc, v_total_delta_skew
      FROM variation_continuous_attr vca
      JOIN effect e ON e.id = vca.effect_id
      JOIN possible_attr pa ON pa.id = e.to_modify_possible_attr_id
     WHERE pa.variant_attr_id = v_variant_attr_id
       AND e.variant_id = (
             SELECT variant_id FROM possible_attr WHERE id = p_possible_attr_id
       )
       AND e.span_id IN (
             SELECT span_id FROM entity_varattr_value WHERE entity_state_id = p_entity_state_id
       )
       AND fn_same_slot(pa.id, p_possible_attr_id);

    -- Compute effective min, max, and mode.
    IF (v_total_delta_conc <> 0 OR v_total_delta_skew <> 0) THEN
        v_eff_min  := (v_min + v_total_delta_mode) * (1 + v_total_delta_conc + v_total_delta_skew);
        v_eff_max  := (v_max + v_total_delta_mode) * (1 + v_total_delta_conc + v_total_delta_skew);
        v_eff_mode := (v_mode + v_total_delta_mode) * (1 + v_total_delta_conc + v_total_delta_skew);
    ELSE
        v_eff_min  := v_min + v_total_delta_mode;
        v_eff_max  := v_max + v_total_delta_mode;
        v_eff_mode := v_mode + v_total_delta_mode;
    END IF;

    IF v_eff_max < v_eff_min THEN
        -- Swap if needed.
        v_result  := v_eff_min;
        v_eff_min := v_eff_max;
        v_eff_max := v_result;
    END IF;

    -- Generate a weighted random value.
    v_result := weighted_random(
                   v_eff_min, 
                   v_eff_max, 
                   v_eff_mode, 
                   COALESCE(v_skew, 0),
                   COALESCE(v_concentration, 0)
               );
    v_result := round(v_result::numeric, v_decimals)::double precision;

    -- Clamp the result to the effective range.
    IF v_result < v_eff_min THEN
        v_clamped_result := v_eff_min;
    ELSIF v_result > v_eff_max THEN
        v_clamped_result := v_eff_max;
    ELSE
        v_clamped_result := v_result;
    END IF;

    -- Select the span matching the computed result.
    -- We join from span -> subvariant_span -> possible_attr and use fn_same_slot so that any 
    -- possible_attr row structurally equivalent to p_possible_attr_id is used.
    IF p_exclude_span_id IS NOT NULL AND p_exclude_span_id <> 0 THEN
        SELECT s.id,
               CASE 
                 WHEN (v_total_delta_conc <> 0 OR v_total_delta_skew <> 0)
                 THEN (s.min_value + v_total_delta_mode) * (1 + v_total_delta_conc + v_total_delta_skew)
                 ELSE (s.min_value + v_total_delta_mode)
               END,
               CASE 
                 WHEN (v_total_delta_conc <> 0 OR v_total_delta_skew <> 0)
                 THEN (s.max_value + v_total_delta_mode) * (1 + v_total_delta_conc + v_total_delta_skew)
                 ELSE (s.max_value + v_total_delta_mode)
               END
          INTO v_chosen_span_id, dummy_eff_min, dummy_eff_max
          FROM span s
          JOIN subvariant_span ss ON ss.span_id = s.id
          JOIN possible_attr pa ON pa.subvariant_span_id = ss.id
         WHERE s.attribute_id = v_attribute_id
           AND s.type = 'continuous'
           AND s.id <> p_exclude_span_id
           AND fn_same_slot(pa.id, p_possible_attr_id)
           AND (CASE 
                   WHEN (v_total_delta_conc <> 0 OR v_total_delta_skew <> 0)
                   THEN (s.min_value + v_total_delta_mode) * (1 + v_total_delta_conc + v_total_delta_skew)
                   ELSE (s.min_value + v_total_delta_mode)
                END) <= v_clamped_result
           AND (CASE 
                   WHEN (v_total_delta_conc <> 0 OR v_total_delta_skew <> 0)
                   THEN (s.max_value + v_total_delta_mode) * (1 + v_total_delta_conc + v_total_delta_skew)
                   ELSE (s.max_value + v_total_delta_mode)
                END) > v_clamped_result
         LIMIT 1;
    ELSE
        SELECT s.id,
               CASE 
                 WHEN (v_total_delta_conc <> 0 OR v_total_delta_skew <> 0)
                 THEN (s.min_value + v_total_delta_mode) * (1 + v_total_delta_conc + v_total_delta_skew)
                 ELSE (s.min_value + v_total_delta_mode)
               END,
               CASE 
                 WHEN (v_total_delta_conc <> 0 OR v_total_delta_skew <> 0)
                 THEN (s.max_value + v_total_delta_mode) * (1 + v_total_delta_conc + v_total_delta_skew)
                 ELSE (s.max_value + v_total_delta_mode)
               END
          INTO v_chosen_span_id, dummy_eff_min, dummy_eff_max
          FROM span s
          JOIN subvariant_span ss ON ss.span_id = s.id
          JOIN possible_attr pa ON pa.subvariant_span_id = ss.id
         WHERE s.attribute_id = v_attribute_id
           AND s.type = 'continuous'
           AND fn_same_slot(pa.id, p_possible_attr_id)
           AND (CASE 
                   WHEN (v_total_delta_conc <> 0 OR v_total_delta_skew <> 0)
                   THEN (s.min_value + v_total_delta_mode) * (1 + v_total_delta_conc + v_total_delta_skew)
                   ELSE (s.min_value + v_total_delta_mode)
                END) <= v_clamped_result
           AND (CASE 
                   WHEN (v_total_delta_conc <> 0 OR v_total_delta_skew <> 0)
                   THEN (s.max_value + v_total_delta_mode) * (1 + v_total_delta_conc + v_total_delta_skew)
                   ELSE (s.max_value + v_total_delta_mode)
                END) > v_clamped_result
         LIMIT 1;
    END IF;

    chosen_span_id := v_chosen_span_id;
    chosen_value   := v_clamped_result;
    RETURN NEXT;
    RETURN;
END;
$$;


Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Generate Entity State

In [1150]:
%%sql

CREATE OR REPLACE PROCEDURE generate_entity_state(
    IN p_entity_name VARCHAR,
    IN p_time        DOUBLE PRECISION
)
LANGUAGE plpgsql
AS $$
DECLARE
    v_entity_id             INTEGER;
    v_root_variant_id       INTEGER;
    v_entity_state_id       INTEGER;
    v_is_regenerate         BOOLEAN;
    v_current_variant       INTEGER;
    v_trigger_ss           INTEGER;  -- the subvariant_span that triggered this variant (NULL for top-level)
    v_parent_possible_attr_id INTEGER;  -- parent's possible_attr id (NULL for top-level)
    cur_va_id               INTEGER;
    cur_attr_type           TEXT;
    v_subvariant_span_id  INTEGER;
    v_existing_evav_id      INTEGER;
    v_existing_span_id      INTEGER;
    v_lock_count            INTEGER := 0;
    v_new_span_id           INTEGER;
    v_new_numeric           DOUBLE PRECISION;
    v_used_span_id          INTEGER;
    v_sub_variant_id        INTEGER;
    v_attr_counter          INTEGER := 0;
    v_queue_id              INTEGER;  -- for stack (LIFO) ordering
    cur_possible_attr_id     INTEGER;  -- resolved unique possible_attr id for this combination
    rec                     RECORD;
BEGIN
    -- Look up the entity by name.
    SELECT id, variant_id
      INTO v_entity_id, v_root_variant_id
      FROM entity
     WHERE name = p_entity_name;
    IF v_entity_id IS NULL THEN
       RAISE EXCEPTION 'No entity found with name=%', p_entity_name;
    END IF;

    -- Check if an entity_state already exists for this entity at the given time.
    SELECT id 
      INTO v_entity_state_id
      FROM entity_state
     WHERE entity_id = v_entity_id
       AND "time" = p_time
     LIMIT 1;
    IF v_entity_state_id IS NOT NULL THEN
        v_is_regenerate := TRUE;
    ELSE
        INSERT INTO entity_state(entity_id, "time")
        VALUES (v_entity_id, p_time)
        RETURNING id INTO v_entity_state_id;
        v_is_regenerate := FALSE;
    END IF;

    -- Create a temporary queue that holds variants to process.
    -- Now the queue includes:
    --   variant_id: the variant whose attributes will be generated.
    --   subvariant_span_id: the span that triggered this variant (may be NULL).
    --   parent_possible_attr_id: the parent's possible_attr id (NULL for top-level).
    DROP TABLE IF EXISTS _variant_queue;
    CREATE TEMPORARY TABLE _variant_queue (
        queue_id serial PRIMARY KEY,
        variant_id INTEGER,
        subvariant_span_id INTEGER,
        parent_possible_attr_id INTEGER
    ) ON COMMIT DROP;
    -- Top-level variant: no triggering span and no parent.
    INSERT INTO _variant_queue (variant_id, subvariant_span_id, parent_possible_attr_id)
      VALUES (v_root_variant_id, NULL, NULL);

    -- Temporary table to hold the current variant's attributes.
    DROP TABLE IF EXISTS _variant_attrs;
    CREATE TEMPORARY TABLE _variant_attrs (
        va_id    INTEGER,
        attr_type TEXT
    ) ON COMMIT DROP;

    -- Process the variant queue (LIFO for depth-first traversal).
    LOOP
        SELECT queue_id, variant_id, subvariant_span_id, parent_possible_attr_id
          INTO v_queue_id, v_current_variant, v_trigger_ss, v_parent_possible_attr_id
          FROM _variant_queue
         ORDER BY queue_id DESC
         LIMIT 1;
        EXIT WHEN NOT FOUND;
        DELETE FROM _variant_queue WHERE queue_id = v_queue_id;

        -- Load the attributes for the current variant.
        TRUNCATE _variant_attrs;
        INSERT INTO _variant_attrs (va_id, attr_type)
          SELECT va.id, a.type
            FROM variant_attr va
            JOIN attribute a ON a.id = va.attribute_id
           WHERE va.variant_id = v_current_variant;

        v_attr_counter := 0;
        FOR rec IN
            SELECT va_id, attr_type FROM _variant_attrs
        LOOP
            cur_va_id     := rec.va_id;
            cur_attr_type := rec.attr_type;
            v_attr_counter := v_attr_counter + 1;

            -- Determine the subvariant_span for this attribute.
            -- If a triggering span was provided, simply use it.
            IF v_trigger_ss IS NOT NULL THEN
                v_subvariant_span_id := v_trigger_ss;
            ELSE
                v_subvariant_span_id := NULL;
            END IF;

            -- Resolve (or create) the unique possible_attr record for this combination.
            -- Uniqueness is defined by (parent_possible_attr_id, subvariant_span_id, variant_attr_id).
            SELECT id
              INTO cur_possible_attr_id
              FROM possible_attr
             WHERE variant_attr_id = cur_va_id
               AND ((parent_possible_attr_id IS NULL AND v_parent_possible_attr_id IS NULL)
                    OR parent_possible_attr_id = v_parent_possible_attr_id)
               AND ((subvariant_span_id IS NULL AND v_subvariant_span_id IS NULL)
                    OR subvariant_span_id = v_subvariant_span_id)
             LIMIT 1;
            IF NOT FOUND THEN
                INSERT INTO possible_attr (variant_attr_id, subvariant_span_id, parent_possible_attr_id)
                VALUES (cur_va_id, v_subvariant_span_id, v_parent_possible_attr_id)
                RETURNING id INTO cur_possible_attr_id;
            END IF;

            -- Process or update the entity_varattr_value for this resolved possible_attr.
            IF v_is_regenerate THEN
                SELECT id, span_id
                  INTO v_existing_evav_id, v_existing_span_id
                  FROM entity_varattr_value
                 WHERE entity_state_id = v_entity_state_id
                   AND possible_attr_id = cur_possible_attr_id
                 LIMIT 1;
            ELSE
                v_existing_evav_id := NULL;
                v_existing_span_id := NULL;
            END IF;

            IF v_is_regenerate AND v_existing_evav_id IS NOT NULL THEN
                SELECT count(*) INTO v_lock_count
                  FROM evav_lock
                 WHERE locked_evav_id = v_existing_evav_id;
                IF v_lock_count > 0 THEN
                    v_used_span_id := v_existing_span_id;
                ELSE
                    IF cur_attr_type = 'discrete' THEN
                        v_new_span_id := roll_discrete_varattr(cur_possible_attr_id, v_entity_state_id, 0);
                        UPDATE entity_varattr_value
                           SET span_id = v_new_span_id
                         WHERE id = v_existing_evav_id;
                        v_used_span_id := v_new_span_id;
                    ELSE
                        SELECT t.chosen_span_id, t.chosen_value
                          INTO v_new_span_id, v_new_numeric
                          FROM roll_continuous_varattr(cur_possible_attr_id, v_entity_state_id, 0) t
                         LIMIT 1;
                        UPDATE entity_varattr_value
                           SET span_id = v_new_span_id,
                               numeric_value = v_new_numeric
                         WHERE id = v_existing_evav_id;
                        v_used_span_id := v_new_span_id;
                    END IF;
                END IF;
            ELSE
                IF cur_attr_type = 'discrete' THEN
                    v_new_span_id := roll_discrete_varattr(cur_possible_attr_id, v_entity_state_id, 0);
                    INSERT INTO entity_varattr_value(
                        entity_state_id,
                        numeric_value,
                        span_id,
                        possible_attr_id
                    ) VALUES (
                        v_entity_state_id,
                        NULL,
                        v_new_span_id,
                        cur_possible_attr_id
                    );
                    v_used_span_id := v_new_span_id;
                ELSE
                    SELECT t.chosen_span_id, t.chosen_value
                      INTO v_new_span_id, v_new_numeric
                      FROM roll_continuous_varattr(cur_possible_attr_id, v_entity_state_id, 0) t
                     LIMIT 1;
                    INSERT INTO entity_varattr_value(
                        entity_state_id,
                        numeric_value,
                        span_id,
                        possible_attr_id
                    ) VALUES (
                        v_entity_state_id,
                        v_new_numeric,
                        v_new_span_id,
                        cur_possible_attr_id
                    );
                    v_used_span_id := v_new_span_id;
                END IF;
            END IF;

            -- If the chosen span activates a sub–variant, enqueue that variant.
            -- The sub–variant is defined by the subvariant_span that produced the used span.
            -- The new queue row carries the current possible_attr as the parent.
            SELECT variant_id
              INTO v_sub_variant_id
              FROM subvariant_span
             WHERE id = v_used_span_id
             LIMIT 1;
            IF v_sub_variant_id IS NOT NULL THEN
                INSERT INTO _variant_queue (variant_id, subvariant_span_id, parent_possible_attr_id)
                VALUES (v_sub_variant_id, v_used_span_id, cur_possible_attr_id)
                ON CONFLICT DO NOTHING;
            END IF;
        END LOOP;
    END LOOP;

    DROP TABLE IF EXISTS _variant_attrs;
    DROP TABLE IF EXISTS _variant_queue;
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Get Entity State Details

In [1151]:
%%sql

CREATE OR REPLACE FUNCTION get_entity_state(
    p_entity_name VARCHAR,
    p_time        DOUBLE PRECISION
)
RETURNS TABLE (
    "index"       BIGINT,
    "address"     TEXT,
    activates     JSON,
    effected_by   JSON,
    numeric_value DOUBLE PRECISION,
    span          VARCHAR,
    locked        BOOLEAN
)
LANGUAGE plpgsql
AS $$
BEGIN
  RETURN QUERY
  WITH RECURSIVE entity_info AS (
      -- Retrieve the entity_state (by matching the entity name and time) and its variant.
      SELECT es.id AS entity_state_id,
             e.variant_id
      FROM entity_state es
      JOIN entity e ON e.id = es.entity_id
      WHERE e.name = p_entity_name
        AND es."time" = p_time
  ),
  base AS (
      -- Base step: select all top-level possible_attr records (those with no parent)
      -- for the variant corresponding to the entity state.
      SELECT 
          pa.id AS possible_attr_id,
          va.name AS attribute_name,
          va.causation_index,
          ARRAY[va.causation_index] AS flattened_path,
          jsonb_build_array(
             CASE 
               WHEN s.label IS NOT NULL 
                 THEN jsonb_build_array(va.name, s.label)
               ELSE jsonb_build_array(va.name)
             END
          ) AS agg_label,
          ev.id AS evav_id,
          ev.numeric_value,
          ev.span_id
      FROM possible_attr pa
      JOIN variant_attr va 
        ON va.id = pa.variant_attr_id
      JOIN entity_info ei 
        ON va.variant_id = ei.variant_id
      JOIN entity_varattr_value ev 
        ON ev.possible_attr_id = pa.id 
       AND ev.entity_state_id = ei.entity_state_id
      LEFT JOIN subvariant_span ss 
        ON ss.id = pa.subvariant_span_id
      LEFT JOIN span s 
        ON s.id = ss.span_id
      WHERE pa.parent_possible_attr_id IS NULL
  ),
  recursive_flattened AS (
      -- Start with the base nodes…
      SELECT * FROM base
      UNION ALL
      -- …and then for each node, recursively join its children.
      SELECT 
          child.possible_attr_id,
          child.attribute_name,
          child.causation_index,
          rf.flattened_path || child.causation_index AS flattened_path,
          rf.agg_label || jsonb_build_array(
             CASE 
               WHEN child_span.label IS NOT NULL 
                 THEN jsonb_build_array(child.attribute_name, child_span.label)
               ELSE jsonb_build_array(child.attribute_name)
             END
          ) AS agg_label,
          child.evav_id,
          child.numeric_value,
          child.span_id
      FROM (
          -- Get child nodes: all possible_attr records that have a non-null parent.
          SELECT 
              pa.id AS possible_attr_id,
              va.name AS attribute_name,
              va.causation_index,
              pa.parent_possible_attr_id,
              ev.id AS evav_id,
              ev.numeric_value,
              ev.span_id,
              pa.subvariant_span_id
          FROM possible_attr pa
          JOIN variant_attr va 
            ON va.id = pa.variant_attr_id
          JOIN entity_varattr_value ev 
            ON ev.possible_attr_id = pa.id 
           AND ev.entity_state_id = (SELECT entity_state_id FROM entity_info)
          WHERE pa.parent_possible_attr_id IS NOT NULL
      ) child
      JOIN recursive_flattened rf 
         ON child.parent_possible_attr_id = rf.possible_attr_id
      LEFT JOIN subvariant_span ss_child 
         ON ss_child.id = child.subvariant_span_id
      LEFT JOIN span child_span 
         ON child_span.id = ss_child.span_id
  ),
  ordered AS (
      -- Order the flattened nodes by the flattened_path and assign a sequential overall_index.
      SELECT 
          row_number() OVER (ORDER BY rf.flattened_path) AS overall_index,
          rf.agg_label,
          rf.possible_attr_id AS resolved_id,
          rf.flattened_path,
          rf.evav_id,
          rf.numeric_value,
          rf.span_id
      FROM recursive_flattened rf
  )
  SELECT 
      o.overall_index AS "index",
      o.agg_label::text AS "address",
      (
         SELECT COALESCE(json_agg(e.name ORDER BY e.name), '[]'::json)
         FROM effect e
         WHERE e.activating_possible_attr_id = o.resolved_id
      ) AS activates,
      (
         SELECT COALESCE(json_agg(e.name ORDER BY e.name), '[]'::json)
         FROM effect e
         WHERE e.to_modify_possible_attr_id = o.resolved_id
      ) AS effected_by,
      o.numeric_value,
      (
         SELECT s.label
         FROM span s
         WHERE s.id = o.span_id
      ) AS span,
      EXISTS (
         SELECT 1 FROM evav_lock l 
         WHERE l.locked_evav_id = o.evav_id
      ) AS locked
  FROM ordered o
  ORDER BY o.overall_index;
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Generate Entity Group

In [1152]:
%%sql

CREATE OR REPLACE PROCEDURE generate_entity_group(
    p_variant_name         VARCHAR,
    p_group_name           VARCHAR,
    p_entity_name_template VARCHAR,
    p_num_entities         INTEGER
)
LANGUAGE plpgsql
AS $$
DECLARE
    v_variant_id       INTEGER;
    v_entity_group_id  INTEGER;
    v_entity_id        INTEGER;
    v_idx              INTEGER;
    v_entity_name      VARCHAR(255);
BEGIN
    -- Look up the variant id using the variant name.
    SELECT id
      INTO v_variant_id
      FROM variant
     WHERE name = p_variant_name;
    IF v_variant_id IS NULL THEN
        RAISE EXCEPTION 'No variant found with name=%', p_variant_name;
    END IF;

    -- Create a new entity group.
    INSERT INTO entity_group (name)
    VALUES (p_group_name)
    RETURNING id INTO v_entity_group_id;

    FOR v_idx IN 1..p_num_entities LOOP
        v_entity_name := format(p_entity_name_template, v_idx);

        -- Create a new entity using the looked-up variant id.
        INSERT INTO entity (variant_id, name)
        VALUES (v_variant_id, v_entity_name)
        RETURNING id INTO v_entity_id;

        INSERT INTO entity_group_link (entity_id, entity_group_id)
        VALUES (v_entity_id, v_entity_group_id);

        -- Call generate_entity_state using the new entity name and time (0 in this example).
        CALL generate_entity_state(v_entity_name, 0);
    END LOOP;
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Delete Entity Group

In [1153]:
%%sql

CREATE OR REPLACE PROCEDURE delete_entity_group(
    p_group_name      VARCHAR,
    p_delete_entities BOOLEAN DEFAULT false
)
LANGUAGE plpgsql
AS $$
DECLARE
    v_entity_group_id INTEGER;
BEGIN
    -- Retrieve the entity group id based on the given name.
    SELECT id INTO v_entity_group_id
    FROM entity_group
    WHERE name = p_group_name;

    IF NOT FOUND THEN
        RAISE NOTICE 'Entity group "%" not found.', p_group_name;
        RETURN;
    END IF;

    -- If the flag is true, delete all entities that are linked to this group.
    IF p_delete_entities THEN
        DELETE FROM entity
        WHERE id IN (
            SELECT entity_id
            FROM entity_group_link
            WHERE entity_group_id = v_entity_group_id
        );
    END IF;

    -- Delete the entity group. The ON DELETE CASCADE on the foreign key
    -- in entity_group_link will automatically remove any associated links.
    DELETE FROM entity_group
    WHERE id = v_entity_group_id;

    RAISE NOTICE 'Entity group "%" deleted successfully.', p_group_name;
END;
$$;


Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Get Entity Group Details

In [1154]:
%%sql

CREATE OR REPLACE FUNCTION get_entity_group_at_time(
    p_group_name VARCHAR,
    p_time       DOUBLE PRECISION
)
RETURNS TEXT
LANGUAGE sql
AS $$
  WITH eg AS (
    -- Find the entity group by its unique name.
    SELECT id
    FROM entity_group
    WHERE name = p_group_name
  ),
  ent AS (
    -- Get all entities in the group.
    SELECT e.id, e.name
    FROM entity e
    JOIN entity_group_link egl ON e.id = egl.entity_id
    WHERE egl.entity_group_id = (SELECT id FROM eg)
  ),
  est AS (
    -- For each entity, get its entity_state at the given time.
    SELECT e.id AS entity_id, e.name AS entity_name, es.id AS entity_state_id
    FROM ent e
    JOIN entity_state es ON es.entity_id = e.id
    WHERE es."time" = p_time
  ),
  state_details AS (
    -- For each entity state, call get_entity_state to obtain the state rows.
    -- Instead of aggregating to an array, we aggregate to a JSON object
    -- keyed by the state's address (e.g. "[["Gender"]]") and only include
    -- the desired fields in the inner object.
    SELECT 
      est.entity_name,
      jsonb_object_agg(
        vs.address,
        jsonb_strip_nulls(
          jsonb_build_object(
            'numeric_value', vs.numeric_value,
            'span', vs.span,
            'activates', vs.activates,
            'effected_by', vs.effected_by
          )
          ||
          CASE 
            WHEN vs.locked THEN jsonb_build_object('locked', vs.locked)
            ELSE '{}'::jsonb
          END
        )
      ) AS details
    FROM est
    CROSS JOIN LATERAL (
      SELECT *
      FROM get_entity_state(est.entity_name, p_time)
    ) AS vs
    GROUP BY est.entity_name
  )
  -- Aggregate each entity’s JSON object under its name.
  SELECT jsonb_pretty(
           COALESCE(
             jsonb_object_agg(entity_name, details),
             '{}'::jsonb
           )
         )
  FROM state_details;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Set Span as Sub-Variant Activating Span

In [1155]:
%%sql

CREATE OR REPLACE PROCEDURE set_subvariant_span(
    p_span_id INT,
    p_variant_id INT
)
LANGUAGE plpgsql
AS $$
BEGIN
  INSERT INTO subvariant_span(span_id, variant_id)
  VALUES (p_span_id, p_variant_id);
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## resolve_variant_path

In [1156]:
%%sql

CREATE OR REPLACE FUNCTION resolve_variant_path(
    p_variant_id INT,
    p_path JSON
) RETURNS INT
LANGUAGE plpgsql
AS $$
DECLARE
    v_chain_length         INT;
    v_current_variant      INT := p_variant_id;
    v_parent_possible_attr  INT := NULL;
    v_i                    INT;
    v_step                 JSON;
    v_attr_name            TEXT;
    v_span_label           TEXT;
    v_variant_attr_id INT;
    v_subvariant_span_id INT;
    v_sub_variant          INT;
    v_possible_attr_id      INT;
BEGIN
    v_chain_length := json_array_length(p_path);
    IF v_chain_length = 0 THEN
       RAISE EXCEPTION 'Variant path is empty';
    END IF;
    
    FOR v_i IN 0 .. v_chain_length - 1 LOOP
        -- Extract the tuple [attribute name, span label] at index v_i.
        v_step := p_path->v_i;
        v_attr_name := v_step->>0;
        v_span_label := v_step->>1;
        
        -- Look up the variant_attr by name in the current variant.
        SELECT id
          INTO v_variant_attr_id
          FROM variant_attr
         WHERE variant_id = v_current_variant
           AND name = v_attr_name
         ORDER BY id
         LIMIT 1;
        IF v_variant_attr_id IS NULL THEN
            RAISE EXCEPTION 'Variant attribute "%" not found in variant % at path index %',
                v_attr_name, v_current_variant, v_i;
        END IF;
        
        -- Look up the subvariant_span record by joining with span using the span label.
        -- Restrict the lookup to the current variant.
        SELECT ss.id, ss.variant_id
          INTO v_subvariant_span_id, v_sub_variant
          FROM subvariant_span ss
          JOIN span s ON s.id = ss.span_id
         WHERE s.label = v_span_label
           AND ss.variant_id = v_current_variant
         ORDER BY ss.id
         LIMIT 1;
        IF v_subvariant_span_id IS NULL THEN
            RAISE EXCEPTION 'No span with label "%" found for variant attribute "%" at path index %',
                v_span_label, v_attr_name, v_i;
        END IF;
        
        -- Resolve (or create) the unique possible_attr record for this step.
        SELECT id
          INTO v_possible_attr_id
          FROM possible_attr
         WHERE variant_attr_id = v_variant_attr_id
           AND subvariant_span_id = v_subvariant_span_id
           AND ((parent_possible_attr_id IS NULL AND v_parent_possible_attr IS NULL)
                OR parent_possible_attr_id = v_parent_possible_attr)
         LIMIT 1;
        IF v_possible_attr_id IS NULL THEN
            INSERT INTO possible_attr (variant_attr_id, subvariant_span_id, parent_possible_attr_id)
            VALUES (v_variant_attr_id, v_subvariant_span_id, v_parent_possible_attr)
            RETURNING id INTO v_possible_attr_id;
        END IF;
        
        -- If not at the final step, a sub-variant must be activated.
        IF v_i < v_chain_length - 1 THEN
            IF v_sub_variant IS NULL THEN
                RAISE EXCEPTION 'Chain impossible: attribute "%" with span label "%" at path index % does not activate a sub-variant',
                    v_attr_name, v_span_label, v_i;
            END IF;
            v_current_variant := v_sub_variant;
        END IF;
        
        -- Set up for the next iteration.
        v_parent_possible_attr := v_possible_attr_id;
    END LOOP;
    
    RETURN v_parent_possible_attr;
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## get_possible_attr_chain

In [1157]:
%%sql

CREATE OR REPLACE FUNCTION get_possible_attr_chain(
    p_possible_attr_id INT
) RETURNS TABLE(level INT, variant_attr_id INT, causation_index INT)
LANGUAGE plpgsql
AS $$
BEGIN
RETURN QUERY WITH RECURSIVE chain(lev, vav_id, va_id, ci, parent) AS (
    -- Start at the leaf: assign level = 1.
    SELECT 1,
           v.id,
           v.variant_attr_id,
           va.causation_index,
           v.parent_possible_attr_id
      FROM possible_attr v
      JOIN variant_attr va ON va.id = v.variant_attr_id
     WHERE v.id = p_possible_attr_id
    UNION ALL
    -- Walk upward.
    SELECT c.lev + 1,
           v.id,
           v.variant_attr_id,
           va.causation_index,
           v.parent_possible_attr_id
      FROM possible_attr v
      JOIN variant_attr va ON va.id = v.variant_attr_id
      JOIN chain c ON v.id = c.parent
)
-- Reverse the order so that the top-most attribute is first.
SELECT row_number() OVER (ORDER BY lev DESC) AS level,
       va_id AS variant_attr_id,
       ci AS causation_index
  FROM chain
 ORDER BY level;
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## compare_possible_attr_order

In [1158]:
%%sql

CREATE OR REPLACE FUNCTION compare_possible_attr_order(
    p_a INT,
    p_b INT
) RETURNS INT
LANGUAGE plpgsql
AS $$
DECLARE
    rec RECORD;
BEGIN
    FOR rec IN
        SELECT COALESCE(a.level, b.level) AS level,
               a.causation_index AS a_ci,
               b.causation_index AS b_ci
          FROM get_possible_attr_chain(p_a) a
          FULL OUTER JOIN get_possible_attr_chain(p_b) b USING (level)
         ORDER BY level
    LOOP
        -- If one chain is shorter, treat the missing value as "less".
        IF rec.a_ci IS NULL AND rec.b_ci IS NOT NULL THEN
            RETURN -1;
        ELSIF rec.b_ci IS NULL AND rec.a_ci IS NOT NULL THEN
            RETURN 1;
        ELSIF rec.a_ci < rec.b_ci THEN
            RETURN -1;
        ELSIF rec.a_ci > rec.b_ci THEN
            RETURN 1;
        END IF;
        -- Otherwise they are equal at this level; continue.
    END LOOP;
    RETURN 0;
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Add Effect

In [1159]:
%%sql

CREATE OR REPLACE PROCEDURE add_effect(
    p_variant_id              INT,
    p_effect_name             VARCHAR,
    p_triggering_span_label   VARCHAR,
    p_activating_variant_path JSON,
    p_to_modify_variant_path  JSON
)
LANGUAGE plpgsql
AS $$
DECLARE
    v_activating_possible_attr_id INT;
    v_to_modify_possible_attr_id  INT;
    v_triggering_span_id         INT;
BEGIN
    -- Resolve the terminal nodes for each branch.
    v_activating_possible_attr_id := resolve_variant_path(p_variant_id, p_activating_variant_path);
    v_to_modify_possible_attr_id  := resolve_variant_path(p_variant_id, p_to_modify_variant_path);
    
    -- Look up the triggering span id using the provided label.
    SELECT s.id 
      INTO v_triggering_span_id
      FROM span s
      WHERE s.label = p_triggering_span_label
        AND s.attribute_id = (
            SELECT va.attribute_id
              FROM variant_attr va
              JOIN possible_attr pa ON pa.variant_attr_id = va.id
             WHERE pa.id = v_activating_possible_attr_id
             LIMIT 1
        )
      LIMIT 1;
    IF NOT FOUND THEN
        RAISE EXCEPTION 'Triggering span with label "%" not found', p_triggering_span_label;
    END IF;
    
    -- Insert the new effect.
    INSERT INTO effect(
        name, 
        variant_id, 
        span_id, 
        to_modify_possible_attr_id, 
        activating_possible_attr_id
    )
    VALUES (
        p_effect_name,
        p_variant_id,
        v_triggering_span_id,
        v_to_modify_possible_attr_id,
        v_activating_possible_attr_id
    );
END;
$$;


Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## get_variant_id

In [1160]:
%%sql

CREATE OR REPLACE FUNCTION get_variant_id(
    p_variant_name VARCHAR
) RETURNS INTEGER
LANGUAGE plpgsql
AS $$
DECLARE
    variant_id INTEGER;
BEGIN
    SELECT id
      INTO variant_id
      FROM variant
     WHERE name = p_variant_name;
     
    IF NOT FOUND THEN
        RAISE EXCEPTION 'Variant "%" not found', variant_id;
    END IF;
    
    RETURN variant_id;
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## get_attribute_id

In [1161]:
%%sql

CREATE OR REPLACE FUNCTION get_attribute_id(
    p_attribute_name VARCHAR
) RETURNS INTEGER
LANGUAGE plpgsql
AS $$
DECLARE
    attribute_id INTEGER;
BEGIN
    SELECT id
      INTO attribute_id
      FROM attribute
     WHERE name = p_attribute_name;
     
    IF NOT FOUND THEN
        RAISE EXCEPTION 'Attribute "%" not found', p_attribute_name;
    END IF;
    
    RETURN attribute_id;
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## get_span_id

In [1162]:
%%sql

CREATE OR REPLACE FUNCTION get_span_id(
    p_attribute_name VARCHAR,
    p_span_type      attribute_type,
    p_span_label     VARCHAR
) RETURNS INTEGER
LANGUAGE plpgsql
AS $$
DECLARE
    span_id INTEGER;
BEGIN
    SELECT s.id
      INTO span_id
      FROM span s
      JOIN attribute a ON a.id = s.attribute_id
     WHERE s.label = p_span_label
       AND a.name = p_attribute_name
       AND s.type = p_span_type;

    IF NOT FOUND THEN
        RAISE EXCEPTION 'Span on attribute "%" of type "%" and label "%" not found', p_attribute_name, p_span_type, p_span_label;
    END IF;

    RETURN span_id;
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## get_effect_id

In [1163]:
%%sql

CREATE OR REPLACE FUNCTION get_effect_id(
    p_variant_name   VARCHAR,
    p_effect_name    VARCHAR
) RETURNS INTEGER
LANGUAGE plpgsql
AS $$
DECLARE
    effect_id INTEGER;
BEGIN
    SELECT e.id
      INTO effect_id
      FROM effect e
      JOIN variant vt ON vt.id = e.variant_id
     WHERE vt.name = p_variant_name
       AND e.name = p_effect_name;

    IF NOT FOUND THEN
        RAISE EXCEPTION 'effect "%" in variant "%" not found', p_effect_name, p_variant_name;
    END IF;

    RETURN effect_id;
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Add Parameter Delta Effect to Variation

In [1164]:
%%sql

CREATE OR REPLACE PROCEDURE add_continuous_effect_to_span(
    p_effect_name             VARCHAR,
    p_variant_name            VARCHAR,
    p_triggering_span_label   VARCHAR,
    p_activating_variant_path JSON,
    p_to_modify_variant_path  JSON,
    p_delta_mode              DOUBLE PRECISION,
    p_delta_conc              DOUBLE PRECISION,
    p_delta_skew              DOUBLE PRECISION
)
LANGUAGE plpgsql
AS $$
DECLARE
    v_variant_id                   INT;
    v_activating_possible_attr_id   INT;
    v_to_modify_possible_attr_id    INT;
    v_triggering_span_id           INT;
    v_effect_id                    INT;
BEGIN
    v_variant_id := get_variant_id(p_variant_name);
    v_activating_possible_attr_id := resolve_variant_path(v_variant_id, p_activating_variant_path);
    v_to_modify_possible_attr_id  := resolve_variant_path(v_variant_id, p_to_modify_variant_path);

    -- Resolve the triggering span id using the provided label.
    SELECT s.id 
      INTO v_triggering_span_id
      FROM span s
      WHERE s.label = p_triggering_span_label
        AND s.attribute_id = (
              SELECT va.attribute_id
                FROM variant_attr va
                JOIN possible_attr pa ON pa.variant_attr_id = va.id
               WHERE pa.id = v_activating_possible_attr_id
              LIMIT 1
          )
      LIMIT 1;
    IF NOT FOUND THEN
       RAISE EXCEPTION 'Triggering span with label "%" not found for the activating branch', p_triggering_span_label;
    END IF;

    -- Insert the effect record.
    INSERT INTO effect(
        name, 
        variant_id, 
        span_id, 
        to_modify_possible_attr_id, 
        activating_possible_attr_id
    )
    VALUES (
        p_effect_name,
        v_variant_id,
        v_triggering_span_id,
        v_to_modify_possible_attr_id,
        v_activating_possible_attr_id
    )
    RETURNING id INTO v_effect_id;

    -- Insert the continuous effect details.
    INSERT INTO variation_continuous_attr(effect_id, delta_mode, delta_conc, delta_skew)
    VALUES (v_effect_id, p_delta_mode, p_delta_conc, p_delta_skew);
END;
$$;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

## Add Discrete Effect to Span

In [1165]:
%%sql

CREATE OR REPLACE PROCEDURE add_discrete_effect_to_span(
    p_effect_name             VARCHAR,
    p_variant_name            VARCHAR,
    p_triggering_span_label   VARCHAR,
    p_activating_variant_path JSON,
    p_to_modify_variant_path  JSON,
    p_changes                 JSON  -- Expecting an array of tuples: [[label, weight], ...]
)
LANGUAGE plpgsql
AS $$
DECLARE
    v_variant_id                 INT;
    v_activating_possible_attr_id INT;
    v_to_modify_possible_attr_id  INT;
    v_triggering_span_id         INT;
    v_effect_id                  INT;
    v_attribute_id               INT;
    v_attribute_name             VARCHAR;
    v_span_id                    INT;
    rec                          JSON;
    v_label                      TEXT;
    v_weight                     DOUBLE PRECISION;
BEGIN
    v_variant_id := get_variant_id(p_variant_name);
    v_activating_possible_attr_id := resolve_variant_path(v_variant_id, p_activating_variant_path);
    v_to_modify_possible_attr_id  := resolve_variant_path(v_variant_id, p_to_modify_variant_path);

    -- Resolve the triggering span id using the provided label.
    SELECT s.id 
      INTO v_triggering_span_id
      FROM span s
      WHERE s.label = p_triggering_span_label
        AND s.attribute_id = (
              SELECT va.attribute_id
                FROM variant_attr va
                JOIN possible_attr pa ON pa.variant_attr_id = va.id
               WHERE pa.id = v_activating_possible_attr_id
              LIMIT 1
          )
      LIMIT 1;
    IF NOT FOUND THEN
       RAISE EXCEPTION 'Triggering span with label "%" not found for the activating branch', p_triggering_span_label;
    END IF;

    -- Insert the effect record.
    INSERT INTO effect(
        name, 
        variant_id, 
        span_id, 
        to_modify_possible_attr_id, 
        activating_possible_attr_id
    )
    VALUES (
        p_effect_name,
        v_variant_id,
        v_triggering_span_id,
        v_to_modify_possible_attr_id,
        v_activating_possible_attr_id
    )
    RETURNING id INTO v_effect_id;

    -- Retrieve the attribute associated with the to-modify branch.
    SELECT a.id, a.name
      INTO v_attribute_id, v_attribute_name
      FROM possible_attr pa
      JOIN variant_attr va ON va.id = pa.variant_attr_id
      JOIN attribute a ON a.id = va.attribute_id
     WHERE pa.id = v_to_modify_possible_attr_id
     LIMIT 1;

    --------------------------------------------------------------------
    -- Process each change tuple in p_changes.
    --   * If the span does not exist: create it and mark it as activated.
    --   * If it exists and the weight is 0: mark it as inactive.
    --   * If it exists and the weight is nonzero: add a delta weight.
    --------------------------------------------------------------------
    IF p_changes IS NOT NULL THEN
       FOR rec IN SELECT * FROM json_array_elements(p_changes) LOOP
          -- Each tuple should be [label, weight]
          v_label  := rec->>0;
          v_weight := (rec->>1)::double precision;

          SELECT id 
            INTO v_span_id
            FROM span
           WHERE attribute_id = v_attribute_id
             AND label = v_label
             AND type = 'discrete'
           LIMIT 1;

          IF NOT FOUND THEN
             -- Span does not exist: create it.
             INSERT INTO span(
                 attribute_id, 
                 label, 
                 type, 
                 is_pinned, 
                 weight, 
                 max_value, 
                 min_value
             )
             VALUES (v_attribute_id, v_label, 'discrete', false, v_weight, NULL, NULL)
             RETURNING id INTO v_span_id;

             -- Record that this span is activated by the effect.
             INSERT INTO variation_activated_span(effect_id, span_id)
             VALUES (v_effect_id, v_span_id);
          ELSE
             IF v_weight = 0 THEN
                -- Mark the span as inactive for this effect.
                INSERT INTO variation_inactive_span(effect_id, span_id)
                VALUES (v_effect_id, v_span_id);
             ELSE
                -- Record a delta–weight effect.
                INSERT INTO variation_delta_weight(effect_id, span_id, delta_weight)
                VALUES (v_effect_id, v_span_id, v_weight);
             END IF;
          END IF;
       END LOOP;
    END IF;
END;
$$;


Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

# Inspect Schema

## List Tables

In [1166]:
%%sql

SELECT tablename AS name
FROM pg_catalog.pg_tables
WHERE schemaname NOT IN ('pg_catalog', 'information_schema')
ORDER BY schemaname, tablename;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

18 rows affected.

name
attribute
debug_log
effect
entity
entity_group
entity_group_link
entity_state
entity_varattr_value
evav_lock
possible_attr


## List Procedures

In [1167]:
%%sql

SELECT proname AS name, pg_catalog.pg_get_function_arguments(p.oid) AS arguments
FROM pg_catalog.pg_proc p
JOIN pg_catalog.pg_namespace n ON n.oid = p.pronamespace
WHERE p.prokind = 'p' AND n.nspname = 'public'
ORDER BY n.nspname, proname;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

18 rows affected.

name,arguments
add_cont_span_to_attr,"IN p_attribute_id integer, IN p_label character varying, IN p_new_min double precision, IN p_new_max double precision"
add_continuous_attribute,"IN p_name character varying, IN p_min_value double precision, IN p_mode_value double precision, IN p_max_value double precision, IN p_concentration double precision, IN p_skew double precision, IN p_decimals integer, IN p_units character varying, IN p_spans json DEFAULT NULL::json"
add_continuous_effect_to_span,"IN p_effect_name character varying, IN p_variant_name character varying, IN p_triggering_span_label character varying, IN p_activating_variant_path json, IN p_to_modify_variant_path json, IN p_delta_mode double precision, IN p_delta_conc double precision, IN p_delta_skew double precision"
add_disc_span_to_attr,"IN p_attribute_id integer, IN p_label character varying"
add_discrete_attribute,"IN in_name character varying, IN in_spans json"
add_discrete_effect_to_span,"IN p_effect_name character varying, IN p_variant_name character varying, IN p_triggering_span_label character varying, IN p_activating_variant_path json, IN p_to_modify_variant_path json, IN p_changes json"
add_effect,"IN p_variant_id integer, IN p_effect_name character varying, IN p_triggering_span_label character varying, IN p_activating_variant_path json, IN p_to_modify_variant_path json"
add_entity,"IN in_name character varying, IN in_variant_id integer"
add_variant,"IN in_variant_name character varying, IN in_attr_keys json"
delete_continuous_attribute,IN p_attribute_name character varying


## List Functions

In [1168]:
%%sql

SELECT proname AS name,
       pg_catalog.pg_get_function_arguments(p.oid) AS arguments,
       pg_catalog.pg_get_function_result(p.oid) AS return_type
FROM pg_catalog.pg_proc p
JOIN pg_catalog.pg_namespace n ON n.oid = p.pronamespace
WHERE p.prokind = 'f' AND n.nspname = 'public'
ORDER BY n.nspname, proname;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

22 rows affected.

name,arguments,return_type
check_delete_possible_attr_from_effect,,trigger
check_delete_possible_attr_from_evav,,trigger
compare_possible_attr_order,"p_a integer, p_b integer",integer
debug_log,"p_procedure_name character varying, p_log_message text",void
delete_orphaned_span,,trigger
fn_same_slot,"p_candidate_id integer, p_reference_id integer",boolean
gamma_rng,shape double precision,double precision
get_attribute_id,p_attribute_name character varying,integer
get_continuous_attribute,p_attribute_name character varying,"TABLE(name character varying, type attribute_type, min double precision, mode double precision, max double precision, concentration double precision, skew double precision, units character varying, decimals integer, span_label character varying, span_min double precision, span_max double precision)"
get_discrete_attribute,p_attribute_name character varying,"TABLE(attribute character varying, type attribute_type, span_label character varying, span_weight double precision, is_pinned boolean)"


## List Indices

In [1169]:
%%sql

SELECT i.relname AS index_name, t.relname AS table_name
FROM pg_catalog.pg_index ix
JOIN pg_catalog.pg_class i ON i.oid = ix.indexrelid
JOIN pg_catalog.pg_class t ON t.oid = ix.indrelid
JOIN pg_catalog.pg_namespace n ON n.oid = t.relnamespace
WHERE n.nspname = 'public'
ORDER BY n.nspname, t.relname, i.relname;

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

66 rows affected.

index_name,table_name
attribute_name_key,attribute
attribute_pkey,attribute
idx_attribute_name,attribute
debug_log_pkey,debug_log
effect_pkey,effect
idx_effect_activating_psattr,effect
idx_effect_to_modify_psattr,effect
unique_effect_name_within_variant,effect
entity_name_key,entity
entity_pkey,entity


# Testing

## Gender Attribute

### Delete Gender Attribute

In [1170]:
%%sql

CALL delete_discrete_attribute('Gender');

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

### Add Gender Attribute

In [1171]:
%%sql

call add_discrete_attribute('Gender', '[
    ["Male", 1], 
    ["Female", 1]
]');

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

### View Gender Attribute

In [1172]:
%%sql

SELECT * FROM get_discrete_attribute('Gender');

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

2 rows affected.

attribute,type,span_label,span_weight,is_pinned
Gender,discrete,Female,0.5,False
Gender,discrete,Male,0.5,False


## Hair Color Attribute

### Delete Hair Color Attribute

In [1173]:
%%sql

CALL delete_discrete_attribute('Hair Color');

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

### Add Hair Color Attribute

In [1174]:
%%sql

call add_discrete_attribute('Hair Color', '[
    ["Blonde", 2], 
    ["Brown", 3],
    ["Golden", 2],
    ["Auburn", 2],
    ["Ginger", 1],
    ["Sandy", 2],
    ["Black", 3],
    ["Gray", 2]
]');

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

### View Hair Color Attribute

In [1175]:
%%sql

SELECT * FROM get_discrete_attribute('Hair Color');

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

8 rows affected.

attribute,type,span_label,span_weight,is_pinned
Hair Color,discrete,Ginger,0.058823529411764705,False
Hair Color,discrete,Auburn,0.11764705882352941,False
Hair Color,discrete,Blonde,0.11764705882352941,False
Hair Color,discrete,Golden,0.11764705882352941,False
Hair Color,discrete,Gray,0.11764705882352941,False
Hair Color,discrete,Sandy,0.11764705882352941,False
Hair Color,discrete,Black,0.17647058823529413,False
Hair Color,discrete,Brown,0.17647058823529413,False


## Height Attribute

### Delete Height Attribute

In [1176]:
%%sql

CALL delete_continuous_attribute('Height');

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

### Add Height Attribute

In [1177]:
%%sql

call add_continuous_attribute('Height', 0, 68, 96, 32, 0, 4, 'in.', '[
    ["Short", 60],
    ["Average", 74],
    ["Tall", 96]
]');

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

### View Height Attribute

In [1178]:
%%sql

SELECT * FROM get_continuous_attribute('Height');

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

3 rows affected.

name,type,min,mode,max,concentration,skew,units,decimals,span_label,span_min,span_max
Height,continuous,0.0,68.0,96.0,32.0,0.0,in.,4,Average,60.0,74.0
Height,continuous,0.0,68.0,96.0,32.0,0.0,in.,4,Short,0.0,60.0
Height,continuous,0.0,68.0,96.0,32.0,0.0,in.,4,Tall,74.0,96.0


## Human Variant

### Delete

In [1179]:
%%sql

CALL delete_variant('Human');

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

### Add

In [1180]:
%%sql

CALL add_variant('Human', '[
    "Gender",
    "Hair Color",
    "Height"
]');

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

### Get

In [1181]:
%%sql

SELECT * FROM get_variant_page('Human', 1, 1000);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

3 rows affected.

index,address,activates,effected_by
1,"[[""Gender""]]",[],[]
2,"[[""Hair Color""]]",[],[]
3,"[[""Height""]]",[],[]


## Generate a Group of Humans

### Delete

In [1182]:
%%sql

CALL delete_entity_group('Test Humans 1', true);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

### Create

In [1183]:
%%sql

CALL generate_entity_group('Human', 'Test Humans 1', 'Test Human %s', 10);

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

++
||
++
++

### View

In [1184]:
import json
import pandas as pd
from IPython.display import display, JSON

result = %sql SELECT get_entity_group_at_time('Test Humans 1', 0)
df = result.DataFrame()
json_data = json.loads(df.iloc[0, 0])
print(json.dumps(json_data, indent=4))

Running query in 'postgresql://notebook@localhost:5432/sahuagin'

1 rows affected.

{
    "Test Human 1": {
        "[[\"Gender\"]]": {
            "span": "Male",
            "activates": [],
            "effected_by": []
        },
        "[[\"Height\"]]": {
            "activates": [],
            "effected_by": [],
            "numeric_value": 59.5376
        },
        "[[\"Hair Color\"]]": {
            "span": "Blonde",
            "activates": [],
            "effected_by": []
        }
    },
    "Test Human 2": {
        "[[\"Gender\"]]": {
            "span": "Female",
            "activates": [],
            "effected_by": []
        },
        "[[\"Height\"]]": {
            "activates": [],
            "effected_by": [],
            "numeric_value": 57.3804
        },
        "[[\"Hair Color\"]]": {
            "span": "Auburn",
            "activates": [],
            "effected_by": []
        }
    },
    "Test Human 3": {
        "[[\"Gender\"]]": {
            "span": "Male",
            "activates": [],
            "effected_by": []
        },
    